In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import _dat_v7
import csv_all_v9
import SPLINE_ANALYTICS_V6

from _dat_v7 import dat
from SPLINE_ANALYTICS_V6 import getTFC
from SPLINE_ANALYTICS_V6 import get_lens_pass_fail
from SPLINE_ANALYTICS_V6 import heatMapVals
from csv_all_v9 import csv



# Data file labelframe

In [3]:
def browseDAT():

    csv_labelframe = LabelFrame(file_input_frame, text="Data File " , height = csv_labelframe_height , width = csv_labelframe_width , name = 'csv_frame')
    csv_labelframe.place(x = csv_labelframe_x_position , y= csv_labelframe_y_position )
    
    Dat_file = Label(csv_labelframe,text = "DAT File Path :")
    Dat_file.place(x = Dat_file_x_position, y= Dat_file_y_position )    
        
    Dat_file_name = StringVar()


    Dat_file_entry = Entry(csv_labelframe,textvariable = Dat_file_name, width = csv_dat_file_entrybox_width ,bd = csv_dat_file_entrybox_border , name = "dat_file_path" )
    Dat_file_entry.place(x = Dat_file_entry_x_position ,y= Dat_file_entry_y_position )
        
    
    button_get_DAT = Button(csv_labelframe,text = "Select Files",command = lambda: browseFiles(Dat_file_name) , width = browse_dat_button_width ,bg = browse_dat_button_bg)       # use lambda functions when we need to pass in variables into the button functions
    button_get_DAT.place(x = button_get_DAT_x_position , y = button_get_DAT_y_position)
    


In [4]:
########################################### button function for the select files button to allow users to browse a file (After clicked , Measurement params , config file , input table ? will appear)

def browseFiles(Dat_file_name):
    
#     global counter_clicked
    
#     counter_clicked = 0 
    
    global new_final_df
    global filepath
    global dat_deleted_params
    global rc_deleted_params
    
    rc_deleted_params = []  # refresh the user deleted parameters once they have loaded in a new file 
    
    dat_deleted_params = []  # refresh the user deleted parameters once they have loaded in a new file 
    
    

    filepath = filedialog.askopenfilename(initialdir = os.getcwd(),                  ##### Browses & finds the filepath user selected
                                          title = "Select a File",
                                          filetypes = (("DAT files",
                                                        "*.dat*"),
                                                       ("all files",
                                                        "*.*")))

    if len(filepath) == 0 :
        
        messagebox.showwarning("showwarning", "Warning ! Please Browse for a valid DAT file !" )
        
        Dat_file_name.set('')
        
    else:
        
       
        Dat_file_name.set(filepath)     # place the filepath in the text box
        new_final_df = dat(filepath)    # load in the dat file data
        storing_cleaned_dat_and_csv_files(new_final_df,filepath)   # save all the dat dataframes

        ################## Function is called to display the inital info and 

        measurement_parameter_list,measurement_values_list = get_all_measurement_parameters_and_values()
        
        wafer_window.counter+=1    # count the button click on the browse files button 

        display_intial_info(measurement_parameter_list,measurement_values_list,wafer_window.counter)


In [5]:
######## need to be in the same working directory

def storing_cleaned_dat_and_csv_files(new_final_df,filepath1):
    
    global get_global_Rc_num   # intialized to get the first rc number so that when users create a config file before opeinng the wafer map we can still make it and not have any errors and instead use the current global variables instead of referring to the saved data
    global subfolder_filename
    
    subfolder_filename = filepath1.rsplit('/',1)[1][:-4] + ' dataframes'

    cleaned_dat_directory = os.path.join(save_location, directory_name)
    dat_filename_directory = os.path.join(cleaned_dat_directory, subfolder_filename)
    
    ######### placing csv data into the directories
    
    FILENAME_csv = filepath1.rsplit('/',1)[1][:-4] + '.csv'                                   # To get the csv filename
    print('FILENAME_csv',FILENAME_csv)
    
    path = filepath1.rsplit('/',1)[0]
    
    csv_filepath = os.path.join(path,FILENAME_csv)
    
    try:
        
        print('csv_filepath',csv_filepath)
        print('filepath', filepath1)
        
        csv_data = csv(csv_filepath,new_final_df.final_df,'0')   # initialize class variable for csv.py file (csv_data)

        if not os.path.exists(directory_name):
            os.makedirs(cleaned_dat_directory)

        directory_contents = os.listdir(directory_name)   # find the subdirectories in the directory_name directory

        get_global_Rc_num = new_final_df.rc[0]   # for the making of a config file we will Sjust take the variables of the first rc number and basically give the base configuration for all 

        if subfolder_filename not in directory_contents:
            os.makedirs(dat_filename_directory)


            ######### placing the DAT data into the directories 

            Dat_final_df = new_final_df.final_df
            Dat_final_df.to_csv(os.path.join(dat_filename_directory, 'final_df_dat.csv'),index=False)

            Dat_freqList = pd.DataFrame(new_final_df.freqList)
            Dat_freqList.to_csv(os.path.join(dat_filename_directory, 'freqList_dat.csv'),index=False)

            Dat_rc_df = pd.DataFrame(new_final_df.rc)
            Dat_rc_df.to_csv(os.path.join(dat_filename_directory, 'rc.csv'),index=False)

            Dat_main_header_df = pd.DataFrame(new_final_df.main_header)
            Dat_main_header_df.to_csv(os.path.join(dat_filename_directory, 'main_header.csv'),index=False)

            Dat_ffl_df = pd.DataFrame(new_final_df.ffl_df)
            Dat_ffl_df.to_csv(os.path.join(dat_filename_directory, 'ffl_df.csv'),index=False)

            Dat_camPosDf = pd.DataFrame(new_final_df.camPosDf)
            Dat_camPosDf.to_csv(os.path.join(dat_filename_directory, 'camPosDf.csv'),index=False)



            ################### saving csv data into local folder

            ####### Gets the Pitch , max frequency & optimise frequency

            PMO_df = pd.DataFrame(csv_data.PMO)
            PMO_df.to_csv(os.path.join(dat_filename_directory, 'PMO_df_csv.csv'),index=False)

            ####### get s1 to t9 list

            s1_t9_list_df = pd.DataFrame(csv_data.camComp)   # gets a list of s1 to t9 (mainly for wafer map parameters)
            s1_t9_list_df.to_csv(os.path.join(dat_filename_directory, 's1_t9_list_df_csv.csv'),index=False)


            ####### get a list of steps 

            list_of_steps_df = pd.DataFrame(csv_data.listSteps)
            list_of_steps_df.to_csv(os.path.join(dat_filename_directory, 'list_of_steps_df_csv.csv'),index=False)


        else:

            pass
    
    except FileNotFoundError:

        raise Exception(messagebox.showwarning("showwarning", "Warning ! No Csv file of name found in same folder as DAT file !" ))
        

In [6]:
################ display the measurement parameters , bind the button click with the wafer window , display the settings , display the wafer map button 

def display_intial_info(given_measurement_parameter_list,given_measurement_values_list,counter):
    
    display_Params(given_measurement_parameter_list,given_measurement_values_list)
    
    if counter == 1:
        
        input_table(y_pos,next_y_pos)  # y pos & next y pos are global variables declared 
    
        config_dropdown('None')
    
    wafer_window.bind_all("<Button-1>", lambda event: focus(event,wafer_window))                # bind the focus event to find the toplevel
    
    wafer_button_font = font.Font(family='Helvetica',size=13)
    
    frequency_dropdown_boxes()
    
#     button = tk.Button(file_input_frame, text="Wafer Map", bg='Orange',command=lambda: new_window(),font = wafer_button_font)                
#     button.place(x = wafer_map_button_x_position , y = wafer_map_button_y_position)

    

In [7]:
##### Gets the measurement parameters and values to be displayed 

def get_all_measurement_parameters_and_values():

    measurement_param_df = new_final_df.main_header
    measurement_param_df.columns = ['Parameters','Values']
    measurement_param_list = measurement_param_df['Parameters'].tolist()
    measurement_param_list = [x.strip() for x in measurement_param_list]

    measurement_value_list = measurement_param_df['Values'].tolist()
    
    return (measurement_param_list,measurement_value_list)

In [8]:
############################################################ Measurement parameters box

def display_Params(Parameters_list,value_list):
    
    param_listbox_labelframe = LabelFrame(file_input_frame, text="Measurement Parameters " , height = param_listbox_labelframe_height , width = param_listbox_labelframe_width , name = 'measurement_listbox_frame')
    param_listbox_labelframe.place(x = param_listbox_labelframe_x_position , y= param_listbox_labelframe_y_position)
    
####################################################   Create a frame and a scrollbar

    listbox_frame = Frame(param_listbox_labelframe,width=measurement_listbox_frame_width,height=measurement_listbox_frame_height)
    my_scrollbar = Scrollbar(listbox_frame , orient = VERTICAL)
    
    
###################################################  Creating a Listbox and attaching it to listbox frame window
    
    listbox = Listbox(listbox_frame,width = param_listbox_width , yscrollcommand = my_scrollbar.set , name = "measurement_parameter_list")
    listbox.pack(side="left")
    
    
###################################################  Configure listbox scrollbar

    my_scrollbar.config(command = listbox.yview)
    my_scrollbar.pack(side = "left", fill =  "y")

    
##################################################  Insert elements into the listbox
    
    for params,values in zip(Parameters_list,value_list):
        listbox.insert(END,str(params) + " :  " + str(values))

        
    listbox.bind("<BackSpace>", lambda event: dat_delete_items(event,listbox))

    listbox_frame.place(x = listbox_frame_x_position, y= listbox_frame_y_position)


In [9]:
################################################## deleting of variables from the listbox , takes in a backspace button event


def dat_delete_items(event,listbox):  
    
    try:
        
        deleted_params = (listbox.get(listbox.curselection()).split(":")[0]).strip()      # get the selected parameter to delete 
        selected_checkboxs = listbox.curselection()                                       # get the selected parameter
        dat_deleted_params.append(deleted_params)

        for selected_checkbox in selected_checkboxs[::-1]:
            listbox.delete(selected_checkbox)
    
    except TclError :
        
        messagebox.showwarning("showwarning", "Warning ! Please Select a Measurement Parameter to delete !" )

# DISPLAY RC LISTBOX

In [10]:
####################################### Lens parameters LISTBOX widget initialization

def display_RC_Params(Parameters_list,value_list,given_canvas):
    
#     print('given_canvas',given_canvas)
    

#################################################### Label frame

    rc_param_listbox_labelframe = LabelFrame(given_canvas, text="Lens Parameters " , height = rc_param_listbox_labelframe_height , width = rc_param_listbox_labelframe_width , name = 'rc_param_listbox_labelframe')
    rc_param_listbox_labelframe.place(x = rc_param_listbox_labelframe_x_postion, y= rc_param_listbox_labelframe_y_postion-10)


####################################################   Create a frame and a scrollbar

    rc_listbox_frame = Frame(rc_param_listbox_labelframe,width= rc_listbox_frame_width ,height= rc_listbox_frame_height, name = 'rc_listbox_frame')
    rc_my_scrollbar = Scrollbar(rc_listbox_frame , orient = VERTICAL)   # scrollbar for the listbox
    rc_my_scrollbar2 = Scrollbar(rc_listbox_frame , orient = HORIZONTAL)   # horizontal scrollbar for the listbox


###################################################  Creating a Listbox and attaching it to listbox frame window

    rc_listbox = Listbox(rc_listbox_frame,width = rc_listbox_width , yscrollcommand = rc_my_scrollbar.set ,xscrollcommand = rc_my_scrollbar2.set , name = "rc_listbox" )
    
    ### horizontal scrollbar (we have to place the horizontal scrollbar , then the listbox , then the vertical scrollbar for it to all show up and work)
    rc_my_scrollbar2.config(command = rc_listbox.xview)
    rc_my_scrollbar2.pack(side = "bottom", fill =  "x")
    rc_listbox.pack(side="left")


###################################################  Configure listbox scrollbar

    rc_my_scrollbar.config(command = rc_listbox.yview)
    rc_my_scrollbar.pack(side = "left", fill =  "y")
    



##################################################  Insert elements into the listbox

    for params,values in zip(Parameters_list,value_list):
        rc_listbox_frame.nametowidget('rc_listbox').insert(END,str(params) + " :  " + str(values))


    rc_listbox.bind("<BackSpace>", lambda event: delete_items(event,rc_listbox,given_canvas))

    rc_listbox_frame.place(x = rc_listbox_frame_x_position , y= rc_listbox_frame_y_position-5)
    
    
################################### export to csv button 

#     export_button = Button(rc_param_listbox_labelframe,)
    
    export_button = Button(rc_param_listbox_labelframe,text = "Export Pass Fail Map to CSV",command = lambda: export_to_csv(given_canvas) , width = 25 ,bg = 'Orange')       # use lambda functions when we need to pass in variables into the button functions
    export_button.place(x = 5 , y = 186)


In [11]:
def export_to_csv(given_canvas):
    
    str_window_number = str(given_canvas)
    window_num = str_window_number.split('.!',1)[1].split('.',1)[0]
#     print('window_num',window_num)
    
    toplevel_specific_filepath = wafer_window_subfolder_name[window_num]
#     print('toplevel_specific_filepath',toplevel_specific_filepath)
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
    
    ##### get pass fail 
    
    tfc_val,correct_mtf_limits = mtf_limits_same_window(window_num)

    list_of_tfcs = get_list_of_tfcs(tfc_val,correct_mtf_limits,window_num)   # should be same number of elements as new_final_df.rc
    
    
    rows = [x.rsplit(',')[0] for x in rc_list]
    cols = [x.rsplit(',')[1] for x in rc_list]

    col_max = int(max(cols, key=float)) 
    row_max = int(max(rows, key=float))
#     print('col_max',col_max,'row_max',row_max)
    
    if col_max > row_max:
        
        used_num = col_max+1
        
    elif col_max < row_max:
        
        used_num = row_max+1
        
    else:
        used_num = col_max+1
    
    print(used_num)
    
    df = pd.DataFrame(index = range(used_num), columns = range(used_num))

    
    for i,val in enumerate(list_of_tfcs):
        df.iloc[int(val[0].split(',')[0]),int(val[0].split(',')[1])] = val[-1]
        
#     try:
        
    date_string = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

    df = df.drop(0)
    df = df.drop(columns =[0])
    filename = toplevel_specific_filepath + " Pass Fail Wafer Map " + date_string + '.csv'
    df.to_csv(filename)
        

In [12]:
################################################## deleting of variables from the listbox , takes in a backspace button event

### delete items for the lens parameter listbox

def delete_items(event,rc_listbox,canvas):
    
    try:
        
        selected_params = (rc_listbox.get(rc_listbox.curselection()).split(":")[0]).strip()
        selected_checkboxs = rc_listbox.curselection()

        rc_deleted_params.append(selected_params)

        for selected_checkbox in selected_checkboxs[::-1]:
            rc_listbox.delete(selected_checkbox)
    
    except TclError :
        
        messagebox.showwarning("showwarning", "Warning ! Please Select a Lens Parameter to delete !"  , parent = canvas)

# WAFER MAP COLOR PARAMETERS

In [13]:
############## get the parameters that are numbers for the wafer map parameters to change the wafer map colors(dropdown box)

def get_wafer_map_parameters():
    
#     #print('1 load csv')
    
    global rc_parameters
    
    toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]
    
#     print('toplevel_specific_filepath',toplevel_specific_filepath)
    
    
    final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,toplevel_number)
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
#     print('toplevel_specific_filepath',toplevel_specific_filepath,'toplevel_number::',toplevel_number)
    
    frequency = str(PMO[-1])
    
    wafer_params = []
    
    ####### get a list of parameters 
    
    cleaned_final_Df = new_final_df_final_df.iloc[0].dropna()   # remove alll the nan values
#     print('cleaned_final_Df',cleaned_final_Df)
    
    
    for parameters, val in cleaned_final_Df.iteritems():
        if parameters=='Reason of Fail TAN CAM' or parameters=='Reason of Fail' or parameters=='Reason of Fail SAG CAM' or parameters=='Defocus' or  parameters== frequency or parameters=='[Measurement' or parameters.startswith('LSF') or parameters.startswith('MTF Peaks') or parameters.startswith('Range of Valid EFL'):
            continue 
        try:
            if isinstance(val, float) == True:
                wafer_params.append(parameters)    
        except:
            pass    
            


    wafer_params.extend(s1_t9_list)
    
#     print('wafer_params::',wafer_params)
    
    return wafer_params

In [14]:
######################### Making of the widgets for the wafer map parameters

def wafer_param_dropdown_selection_widgets(window,frame,canvas,given_upper_percentile,given_lower_percentile,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,given_selected_wafer_parameter,col_max,col_min,row_min,row_max):
    
    #print('that is the wafer_param_dropdown_selection_widgets tooltip1::',tooltip_1 )
    
    global wafer_param_dropdown_labelframe   # global because wafer_map_dropdown is called in the load config funtion and this so far cant be passed into that function  (may not actly need to global)
    
    label_fontsize = font.Font(family='Arial',size=9)
    
    wafer_param_dropdown_labelframe = LabelFrame(canvas, text="Wafer Parameters " , height = wafer_param_dropdown_labelframe_height , width = wafer_param_dropdown_labelframe_width , name = 'wafer_param_dropdown_labelframe')
    wafer_param_dropdown_labelframe.place(x = wafer_param_dropdown_labelframe_x_position, y= wafer_param_dropdown_labelframe_y_position-10)
    ##print('wafer_param_dropdown_labelframe', wafer_param_dropdown_labelframe)
    
    wafer_prompt = Label(wafer_param_dropdown_labelframe, text = "Select Parameter :" )  # select parameter text
    wafer_prompt.place(x = wafer_prompt_x_position , y = wafer_prompt_y_position)
    
    ########################## Dropdown box 

    wafer_map_combobox(get_wafer_map_parameters(),given_selected_wafer_parameter)   # create the dropdown box 
    
    
    ########################## Entry box for the wafer parameters labelframe
    
    wafer_upper_lower_percentile(wafer_param_dropdown_labelframe,window,frame,canvas,window_scrollbar,given_upper_percentile,given_lower_percentile,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max)
    
    ########################## Wafer window pass fail map toggle
    
    pass_fail_toggle_combobox(wafer_param_dropdown_labelframe)
    

In [15]:
########################## Wafer parameter Dropdown box 

def wafer_map_combobox(given_list,given_parameter_selected):
    
    global wafer_map_dropdown
    
    
    wafer_map_dropdown = ttk.Combobox(wafer_param_dropdown_labelframe, state="readonly", width = wafer_map_dropdown_width,name = "wafer_map_dropdown")

    if given_parameter_selected in given_list:

        index = given_list.index(given_parameter_selected)
        
    else:
        index = None
    
    wafer_parameter_list = given_list
    wafer_map_dropdown['values'] = wafer_parameter_list

        
    wafer_map_dropdown.current(index)

    
    wafer_map_dropdown.place(x = wafer_map_dropdown_x_position, y = wafer_map_dropdown_y_position)

In [16]:
########################## Wafer parameter Dropdown box 

def pass_fail_toggle_combobox(wafer_param_dropdown_labelframe):
    
#     global pass_fail_prompt
#     global pass_fail_toggle_dropdown
    
    ##### labels for dropdown
    
    pass_fail_prompt = Label(wafer_param_dropdown_labelframe, text = "Toggle Pass Fail Map :" )  # select parameter text
    pass_fail_prompt.place(x = pass_fail_label_x_position , y = pass_fail_label_y_position)
    
    #### dropdown box
    
#     current_var = tk.StringVar()
    
    pass_fail_toggle_dropdown = ttk.Combobox(wafer_param_dropdown_labelframe, state="readonly", width = wafer_map_dropdown_width,name = "pass_fail_toggle_dropdown")
    
    
    pass_fail_list = ['Pass/Fail']
    pass_fail_toggle_dropdown['values'] = pass_fail_list
#     pass_fail_toggle_dropdown.current(0)

    
    pass_fail_toggle_dropdown.place(x = pass_fail_dropdown_x_position, y = pass_fail_dropdown_y_position)
    
    
    ##### bind the selection of a value in the combobox
    ##print('pass fail toplevel_number',toplevel_number)
#     pass_fail_toggle_dropdown.bind('<<ComboboxSelected>>', lambda event: load_in_wafer_map_and_legend(toplevel_number))

In [17]:
def wafer_upper_lower_percentile(wafer_param_dropdown_labelframe,window,frame,canvas,window_scrollbar,given_upper_percentile,given_lower_percentile,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max):
    
#     ##print('this is wafer_upper_lower_percentile :given_upper_percentile ',given_upper_percentile)
#     ##print('this is wafer_upper_lower_percentile :given_lower_percentile ',given_lower_percentile)
    
    upper_limit_label = Label(wafer_param_dropdown_labelframe, text = "Color Scale Highest Percentile :"  )
    upper_limit_label.place(x = upper_limit_label_x_position, y = upper_limit_label_y_position)
    
    lower_limit_label = Label(wafer_param_dropdown_labelframe, text = "Color Scale Lowest Percentile :" )
    lower_limit_label.place(x = lower_limit_label_x_position, y = lower_limit_label_y_position)
    
    upper_limit = Entry(wafer_param_dropdown_labelframe,relief=RIDGE,bd = '2' , name = 'upper_limit' , width = upper_lower_percentile_entrybox_width)                    # Name each box with a new S_positon (S1 to S9)
    upper_limit.place(x = upper_limit_entry_x_position , y = upper_limit_entry_y_position)
    
    upper_limit.insert(0, given_upper_percentile)
    
    lower_limit = Entry(wafer_param_dropdown_labelframe,relief=RIDGE,bd = '2' , name = 'lower_limit' , width = upper_lower_percentile_entrybox_width)                    # Name each box with a new S_positon (S1 to S9)
    lower_limit.place(x = lower_limit_entry_x_position , y = lower_limit_entry_y_position)
    
    lower_limit.insert(0, given_lower_percentile)
    
    confirm_upper_lower_limit = Button(wafer_param_dropdown_labelframe,text = "Update Map ", width = "25" ,bg = 'Orange' , name = 'confirm_upper_lower_limit_button',command = lambda: confirm_upper_lower_limit_button_function(window,frame,canvas,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,wafer_param_dropdown_labelframe,col_max,col_min,row_min,row_max))
    confirm_upper_lower_limit.place(x = confirm_upper_lower_limit_x_position , y =  confirm_upper_lower_limit_y_position)

In [18]:
############################## Check if input value is a integer

def confirm_upper_lower_limit_check(window,given_labelframe):
    
#     ##print('that is the ,' , window)

    highlight_COLOR = "red"
    upper_limit_error = 0
    lower_limit_error = 0
    upper_limit_more_or_less_than_100_0 = 0
    lower_limit_more_or_less_than_100_0 = 0
    
    
    upper_limit_entered = window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').get()
    lower_limit_entered = window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').get()
    
    ##print('this is the window : ',window)
    ##print('upper_limit_entered : ',upper_limit_entered)
    ##print('lower_limit_entered : ',lower_limit_entered)
    
#     ##print('confirm_upper_lower_limit_check',given_labelframe.nametowidget('upper_limit'))
#     ##print('confirm_upper_lower_limit_check', given_labelframe.nametowidget('lower_limit'))
    
        
    
    try:
        
        upper_limit_entered = int(upper_limit_entered)
        #print('upper_limit_entered',upper_limit_entered)
    
    except (ValueError,TypeError):
        
        upper_limit_error += 1
        
    
    try:
        
        lower_limit_entered = int(lower_limit_entered)
    
    except (ValueError,TypeError):
        
        lower_limit_error += 1
        
    ########## check if the upper limit is still a string 
    
    if isinstance(upper_limit_entered, str):
        
        #print('upper_limit_entered',upper_limit_entered,type(upper_limit_entered))
    
        upper_limit_error+=1
    
    elif isinstance(lower_limit_entered, str):
        
        #print('lower_limit_entered',lower_limit_entered,type(lower_limit_entered))
        
        lower_limit_error+=1
        
    else:
        
        #print('upper_limit_entered',upper_limit_entered,type(upper_limit_entered))
        
        ####### check if the limits are above 100 or below 0 
    
        if (upper_limit_entered > 100 or upper_limit_entered < 0 ):

            upper_limit_more_or_less_than_100_0 += 1

        elif (lower_limit_entered > 100 or lower_limit_entered < 0 ):

            lower_limit_more_or_less_than_100_0 += 1

    #     selected_wafer_paramter = wafer_param_dropdown_labelframe.nametowidget("wafer_map_dropdown").get()
    selected_wafer_paramter = window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget("wafer_map_dropdown").get()


    ######### check if there is a selected parameter if not show warning

    if len(selected_wafer_paramter) == 0 :

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').delete(0, END)

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').insert(0,90)

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').delete(0, END)

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').insert(0,10)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Choose a Wafer Parameter or toggle for Pass Fail Map ",parent=window)) 

    elif upper_limit_error > 0 and lower_limit_error > 0:

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Enter a Whole Number in Both Boxes",parent=window)) 

    elif upper_limit_error > 0:

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Enter a Whole Number for the Upper Limit",parent=window)) 

    elif lower_limit_error > 0:

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Enter a Whole Number for the Lower Limit",parent=window)) 


    elif lower_limit_entered > upper_limit_entered:

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Lower Limit entered is higher than Upper Limit",parent=window)) 


    elif upper_limit_more_or_less_than_100_0 > 0 :

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Enter a Number between 1 and 100 for the Upper Limit",parent=window)) 


    elif lower_limit_more_or_less_than_100_0 > 0 :

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').config(highlightthickness=1, highlightbackground= highlight_COLOR)

        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Enter a Number between 1 and 100 for the Lower Limit",parent=window)) 


    else:
        
        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('upper_limit').config(highlightthickness=0)
        
        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget('lower_limit').config(highlightthickness=0)

        return(upper_limit_entered,lower_limit_entered)

In [19]:
#################################### Confirm upper lower button function (check & then change the color if correct)

def confirm_upper_lower_limit_button_function(window,frame,canvas,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height , given_labelframe,col_max,col_min,row_min,row_max):

        
    if window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget("pass_fail_toggle_dropdown").get() == 'Pass/Fail':

        window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget("pass_fail_toggle_dropdown").set('')   # reset the dropdown box
    
        wafer_window_parameters[toplevel_number] = "blank"

        load_in_wafer_map_and_legend(toplevel_number)


    else:
#         ##print()
        
        ######### loading the final df
    
        toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]
        
        new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
        
        final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,toplevel_number)

        
        ######### Load in the color

        upper_limit_entered,lower_limit_entered = confirm_upper_lower_limit_check(window,given_labelframe)

        chosen_wafer_map_parameter = given_labelframe.nametowidget("wafer_map_dropdown").get()
#         #print('this is the chosen_wafer_map_parameter',chosen_wafer_map_parameter)

        wafer_window_parameters[toplevel_number] = chosen_wafer_map_parameter

        ##print('confirm_upper_lower_limit_button_function window',window)
        ##print('confirm_upper_lower_limit_button_function frame',frame)
        ##print('confirm_upper_lower_limit_button_function canvas',canvas)
        ##print('confirm_upper_lower_limit_button_function wafer_frame',wafer_frame)
        ##print('confirm_upper_lower_limit_button_function given_labelframe',given_labelframe)

        ### get the list from the wafer parameter selected
        
        #print('this is tooltip_1::', tooltip_1)
        get_the_correct_tooltip = Pmw.Balloon(wafer_frame)   # reinitialize the tooltip
        #print('get_the_correct_tooltip',get_the_correct_tooltip)
        
        ######### for s1 to t9 values
        
        ### get latest scan for each rc number
        
        scan_dictionary ={i:rc_list.count(i) for i in rc_list}
        
        col = new_final_df_final_df.columns.tolist()
        
        try:
    
            if chosen_wafer_map_parameter in col:

                wafer_map_list = [float(new_final_df_final_df[chosen_wafer_map_parameter].where((new_final_df_final_df['Position'] == rc_list[i]) & (new_final_df_final_df['No. of Scans'] == scan_dictionary[rc_list[i]])).dropna().values.tolist()[0]) for i in range(len(rc_list))]

            else:

                col = new_final_df_final_df.columns.tolist()[new_final_df_final_df.columns.tolist().index('Camera Pos') + 1]

                wafer_map_list = [float(new_final_df_final_df[col].where((new_final_df_final_df['Position'] == rc_list[i]) & (new_final_df_final_df['Camera Pos'] == chosen_wafer_map_parameter) & (new_final_df_final_df['No. of Scans'] == scan_dictionary[rc_list[i]])).dropna().values.tolist()[0]) for i in range(len(rc_list))]
        
        except IndexError:
            
            window.nametowidget("wafer_map_window_container").nametowidget("window_canvas").nametowidget("wafer_param_dropdown_labelframe").nametowidget("wafer_map_dropdown").set('')   # reset the wafer parameters dropdown to be blank
            
            raise Exception(messagebox.showwarning("showwarning", "Warning ! No Data available for selected Parameter ",parent=window))
            

        ####### refersh wafer maps

        wafer_map_legend(col_max,col_min,row_min,row_max,wafer_frame,pixelVirtual,wafer_map_button_width,wafer_map_button_height)

        wafer_map_creation(wafer_map_list,wafer_frame,pixelVirtual,get_the_correct_tooltip,wafer_map_button_width,wafer_map_button_height,window , frame ,canvas,window_scrollbar,upper_limit_entered,lower_limit_entered ,col_max,col_min,row_min,row_max,'No' )
            

In [20]:
def get_correct_dataframes(toplevel_specific_filepath):   # get correct DAT data
    
    ## toplevel_specific_filepath is subfolder filename 
    

    cleaned_dat_directory = os.path.join(save_location, directory_name)
    dat_filename_directory = os.path.join(cleaned_dat_directory, toplevel_specific_filepath)
    
    ### load in final_df
    
    new_final_df_final_df = pd.read_csv(os.path.join(dat_filename_directory, 'final_df_dat.csv'))   # load in final_Df
    
    ### load in freqList
    
    freq_list_df = pd.read_csv(os.path.join(dat_filename_directory, 'freqList_dat.csv'))   # load in freqlist
    freq_list = freq_list_df['0'].values.tolist()
    
    ### load in rc
    
    new_final_df_rc = pd.read_csv(os.path.join(dat_filename_directory, 'rc.csv'))   # load in rc_number
#     #print('new_final_df_rc',new_final_df_rc)
    rc_list = new_final_df_rc['0'].values.tolist()   # list of rc numbers
    
    ### load in ffl_df
    
    new_final_df_ffl_df = pd.read_csv(os.path.join(dat_filename_directory, 'ffl_df.csv'))   # load in ffl_df
    
    ### load in camposdf
    
    new_final_df_camPosDf = pd.read_csv(os.path.join(dat_filename_directory, 'camPosDf.csv'))   # load in camposdf
    
    ### load in main_header
    
    new_final_df_main_header = pd.read_csv(os.path.join(dat_filename_directory, 'main_header.csv'))   # load in main_header
    
    #### load in max number of scan for each rc number
    
#     max_scan_per_rc = pd.read_csv(os.path.join(dat_filename_directory, 'scan_rc_df.csv'))   # load in main_header
    
    
    return(new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list)

In [21]:
def get_correct_csv_dataframes(toplevel_specific_filepath,given_window_number):
    
    ## toplevel_specific_filepath is subfolder filename 
    
    
    get_final_df_csv_name = frequency_csv_for_each_Window[given_window_number] 
    
    
    cleaned_dat_directory = os.path.join(save_location, directory_name)
    dat_filename_directory = os.path.join(cleaned_dat_directory, toplevel_specific_filepath)
    
    ### load in csv final df
    
    final_csv_df = pd.read_csv(os.path.join(dat_filename_directory, get_final_df_csv_name))   # load in final_csv_df
    
    ### Get a list of Pitch , max frequency & optimise frequency
    
    PMO_df = pd.read_csv(os.path.join(dat_filename_directory, 'PMO_df_csv.csv'))   # load in final_csv_df
    PMO = PMO_df['0'].values.tolist()   # list of PMO numbers
    
    ####### Get a s1 to t9 list
    
    s1_t9_list_df = pd.read_csv(os.path.join(dat_filename_directory, 's1_t9_list_df_csv.csv'))   # load in final_csv_df
    s1_t9_list = s1_t9_list_df['0'].values.tolist()   # list of s1 to t9 values     
    
    ####### Get a list of steps 
    
    list_of_steps_df = pd.read_csv(os.path.join(dat_filename_directory, 'list_of_steps_df_csv.csv'))   # load in final_csv_df
    list_of_steps = list_of_steps_df['0'].values.tolist()   # list of PMO numbers
    
    return(final_csv_df,PMO,s1_t9_list,list_of_steps)

In [22]:
def frequency_dropdown_boxes():
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(subfolder_filename)  # Load in dat data that has been saved into the file 
    
    
    select_frequency_labelframe = LabelFrame(file_input_frame, text="Select Frequency " , height = select_frequency_labelframe_height , width = csv_labelframe_width , name = 'select_frequency_labelframe')
    select_frequency_labelframe.place(x = frequency_labelframe_x_position , y= frequency_labelframe_y_position )
#     #print('select_frequency_labelframe',select_frequency_labelframe)
    
    ################################################
    
    ######## dropdown 1 labels
    
    frequency_dropdown_1_label = Label(select_frequency_labelframe,text = "Select Frequency 1 :")
    frequency_dropdown_1_label.place(x = frequency_dropdown_labels_x_position , y= frequency_dropdown_1_labels_y_position)
    
    ####### dropdown 1 (dropdown creation)
    
    frequency_num = tk.StringVar()
    
    frequency_dropdown_1 = ttk.Combobox(select_frequency_labelframe, state="readonly", width = frequency_dropdown_combobox_width, textvariable = frequency_num , name = "frequency_dropdown_1")
    frequency_dropdown_1.place(x = frequency_dropdown_combobox_x_position ,y= frequency_dropdown_1_combobox_y_position)
    
    frequency_dropdown_1.bind('<<ComboboxSelected>>', lambda event: select_frequency_dropdown(event,frequency_dropdown_1))    # bind the freqeuncy dropdown to a onclick event
    
    frequency_dropdown_1['values'] = freq_list
    
    
    #################################################
    
    
    ######## dropdown 2 labels
    
    frequency_dropdown_2_label = Label(select_frequency_labelframe,text = "Select Frequency 2 :")
    frequency_dropdown_2_label.place(x = frequency_dropdown_labels_x_position , y= frequency_dropdown_2_labels_y_position)
    
    ####### dropdown 2 (dropdown creation)
    
    frequency_num_2 = tk.StringVar()
    
    frequency_dropdown_2 = ttk.Combobox(select_frequency_labelframe, state="readonly", width = frequency_dropdown_combobox_width, textvariable = frequency_num_2 , name = "frequency_dropdown_2")
    frequency_dropdown_2.place(x = frequency_dropdown_combobox_x_position ,y= frequency_dropdown_2_combobox_y_position)
    
    frequency_dropdown_2.bind('<<ComboboxSelected>>', lambda event: select_frequency_dropdown(event,frequency_dropdown_2))    # bind the freqeuncy dropdown to a onclick event
    
    frequency_dropdown_2['values'] = freq_list
    
    #################################################
    
    
    ######## dropdown 3 labels
    
    frequency_dropdown_3_label = Label(select_frequency_labelframe,text = "Select Frequency 3 :")
    frequency_dropdown_3_label.place(x = frequency_dropdown_labels_x_position , y= frequency_dropdown_3_labels_y_position)
    
    ####### dropdown 3 (dropdown creation)
    
    frequency_num_3 = tk.StringVar()
    
    frequency_dropdown_3 = ttk.Combobox(select_frequency_labelframe, state="readonly", width = frequency_dropdown_combobox_width, textvariable = frequency_num_3 , name = "frequency_dropdown_3")
    frequency_dropdown_3.place(x = frequency_dropdown_combobox_x_position ,y= frequency_dropdown_3_combobox_y_position)
    
    frequency_dropdown_3.bind('<<ComboboxSelected>>', lambda event: select_frequency_dropdown(event,frequency_dropdown_3))    # bind the freqeuncy dropdown to a onclick event
    
    frequency_dropdown_3['values'] = freq_list
    
    
    ################################################
    
    
    ######## Heat map dropdown toggle
    
    
    heat_map_label = Label(select_frequency_labelframe,text = "Toggle Heatmap :")
    heat_map_label.place(x = heat_map_label_x_position , y= heat_map_label_y_position)
    
    heat_map_dropdown = ttk.Combobox(select_frequency_labelframe, state="readonly", width = frequency_dropdown_combobox_width,  name = "heat_map_dropdown")
    heat_map_dropdown.place(x = heat_map_dropdown_x_position ,y= heat_map_dropdown_y_position)
    
    heat_map_dropdown.bind('<<ComboboxSelected>>', lambda event: toggle_heat_map(event))    # bind the freqeuncy dropdown to a onclick event
    
    heat_map_dropdown['values'] = ['Yes']
    

In [23]:
def select_frequency_dropdown(event,dropdown_number):
    
    global final_csv_filename
    
    #### dropdown_number will be a frequency dropdown widget
    
    frequency_number_selected = dropdown_number.get()  # get the frequency number 
#     #print('i am the frequency_number_selected',frequency_number_selected)
    
    
    #### call the csv files & save the csv files to a folder 

    
    cleaned_dat_directory = os.path.join(save_location, directory_name)
    dat_filename_directory = os.path.join(cleaned_dat_directory, subfolder_filename)
        
    ######### placing csv data into the directories

    FILENAME_csv = filepath.rsplit('/',1)[1][:-4] + '.csv'                                   # To get the csv filename
    
    final_csv_filename = 'final_df_csv_'+frequency_number_selected + '.csv'   # we need to store the csv filename for each csv 
    
    path = filepath.rsplit('/',1)[0]
    
    csv_file = os.path.join(path,FILENAME_csv)

    ###### initialize the csv py file 
    
    if final_csv_filename not in os.listdir(dat_filename_directory):
    
        csv_data = csv(csv_file,new_final_df.final_df,frequency_number_selected)   # initialize class variable (csv_data)
        final_csv_df = csv_data.final_df   # gets the full cleaned data of the entire csv file
        final_csv_df.to_csv(os.path.join(dat_filename_directory, final_csv_filename),index=False)

    #### call the wafer map 
    
    new_window()

In [24]:
def toggle_heat_map(event):

    if len(pass_list_dictionary) < 3:
        
        file_input_frame.nametowidget('select_frequency_labelframe').nametowidget('heat_map_dropdown').set('')
        
        messagebox.showwarning("showwarning", "Warning ! Please select at least 3 freqeuncies before selecting a Heat Map  !")
        
    else:
        
        new_window()

# INPUT TABLE (Settings)

In [25]:
################################ Config file dropdown box frame

def config_dropdown(text_file_chosen):
    
    global config_file_dropdown   # need this to refer to the config dropdown (config file dropdown widget name )
    
    global config_labelframe      # need this to refer to the config dropdwon
    
    config_labelframe = LabelFrame(file_input_frame, text="Config File " , height = config_labelframe_height , width = config_labelframe_width , name = "config_labelframe")
    config_labelframe.place(x = config_labelframe_x_position , y = config_labelframe_y_position)
    
    Save_file = Label(config_labelframe,text = "File Path:")
    Save_file.place(x = Save_file_x_position , y= Save_file_y_position)
    
    cwd = os.getcwd()  # probably not needed 
    
    ###################################################################################### config file dropdown

    text_files = []                                                                           # find the text files in the cwd
    for text_file in glob.glob(config_file_type):                 
        text_files.append(text_file)
    
    text_files.insert(0,"Create a new config file")
    
    config_file_dropdown = ttk.Combobox(config_labelframe, state="readonly", width = config_dropdown_width, name = "config_file_dropdown")
    
    if text_file_chosen == 'None':
        
        index = None
        
    else:

#         for text_file in glob.glob(config_file_type):
#             text_files.append(text_file)
            
        correct_text_file = text_file_chosen + config_file_type.split('*',1)[1]
            
        index = text_files.index(correct_text_file)
    
    
    config_file_dropdown['values'] = text_files
    
    config_file_dropdown.current(index)
    
    config_file_dropdown.bind('<<ComboboxSelected>>', lambda event: get_config(event,config_file_dropdown))   # changed to use the global rc number instead of a passed in number as we would just get the first rc number that the user has clicked

    ######################################################################################### buttons & Dropdown placement


    config_file_dropdown.place(x = config_file_dropdown_x_position,y= config_file_dropdown_y_position)                          # place the dropdown bar  

    button_save_input = Button(config_labelframe,text='Update Settings', command= lambda: savedata_overwrite(cwd,sag_tan_tfc), width = button_save_input_width ,bg = button_save_input_bg)
    button_save_input.place(x = button_save_input_x_position , y = button_save_input_y_position )
    
    

In [26]:
######## get the config files

def get_config(event,config_file_dropdown):
    
    global chosen_config 
    
    chosen_config = config_file_dropdown.get()   # get the config that the user has chosen 
    current_working_directory = os.getcwd()  # maybe change this to be a global variable ?
    Save_file_name = current_working_directory + '/' + chosen_config 
    
    
    if chosen_config == 'Create a new config file':
        
        users_choice = messagebox.askquestion("askyesno", "Warning ! Would you like to create a new config file ? ")
        
        if users_choice=='yes':
        
            check_input(sag_tan_tfc)

            config_popup = Tk()
            config_popup.geometry("900x200")

            Save_file_label = Label(config_popup,text = "Enter a Config Filename :")
            Save_file_label.place(x = x_label_start_pos , y= y_height_pos)

            save_file_name = Entry(config_popup, width = save_file_name_width ,bd =  save_file_name_border, name = "save_file_name")
            save_file_name.place(x = x_label_start_pos + 140 , y= y_height_pos)

            button_get_filename = Button(config_popup,text='Confirm filename', command= lambda: get_filename_popup(save_file_name,config_popup), width = button_get_filename_width ,bg = button_get_filename_bg)
            button_get_filename.place(x = x_label_start_pos + 350 , y= y_height_pos)
            
        else:
            config_file_dropdown.set('')
            
            pass
         
######################################################## Load in the data (make a dataframe and split the sections into sepearate dataframes)

    else:       
        
        load_in_selected_config_file(chosen_config,Save_file_name)

In [27]:
def load_in_selected_config_file(chosen_config,Save_file_name):


    with open(Save_file_name,'r') as reader:
        config_list = reader.readlines()
        new_df = pd.DataFrame(config_list)[0].str.rsplit(":",1, expand=True)
        new_df = new_df.replace('\n','', regex=True) # combine with previous line
        new_df.columns = ['Parameters', 'Values']

        ####### load the config files into a big df then split based on the indexes of the sections '[MTF Specifications]' ,'[Lens Parameters]','[Measurement Parameters]'

        mtf_lens_measurement_df = new_df.loc[(new_df['Parameters'].str.startswith(mtf_specs)) | (new_df['Parameters'].str.startswith(lens)) | (new_df['Parameters'].str.startswith(measurement))]  #add in wafer map display here       # only contains the header of each section ([MTF Specifications],[Lens Parameters])
        index_list = mtf_lens_measurement_df.index.tolist()

        df_mtf = new_df.iloc[index_list[0]:index_list[1]]
        df_lens = new_df.iloc[index_list[1]:index_list[2]]
        df_measurement = new_df.iloc[index_list[2]:]                      

        ############ split into seperate functions from here


        df_mtf_params = df_mtf['Parameters'].values.tolist()[1:]                              # find out the parameters so we can use the s1 to t9 value to refer to the correct entry box name (s1-t9)
        df_mtf_values = df_mtf['Values'].values.tolist()[1:]

###################################### finds all the mtf values parameters (s1 to t9 values)

        df_mtf_values = [x.strip() for x in df_mtf_values]                                    # strip all the whitespace away
        ##print('df_mtf_values',df_mtf_values)
        df_mtf_values_new = [df_mtf_values[x].split('.')[0] for x in range(len(df_mtf_values)) if x+1 < len(df_mtf_values) ]     # remove the .0 from behind the number only for numbers from s1 to t9 and not tfc
        df_mtf_values_new.append(df_mtf_values[-1])
        ##print('new df_mtf_values',df_mtf_values_new)

###################################### finds all the lens parameters

        df_lens_params = df_lens['Parameters'].values.tolist()[1:]
        df_lens_params = [x.strip() for x in df_lens_params]                                  # list of params names


###################################### finds all the Measurement parameters

        df_measurement_params = df_measurement['Parameters'].values.tolist()[1:]
        df_measurement_params = [x.strip() for x in df_measurement_params]                    # list of measurement names


###################################### Load in the s1 to t9 values into the settings 

        for s1_t9,nums in zip(df_mtf_params,df_mtf_values_new):                                   # based on the product number , we will find the s1 all the way to tfc and then place the values into the inputbox
            input_table_labelframe.nametowidget(s1_t9).delete('0',END)
            input_table_labelframe.nametowidget(s1_t9).insert(END,nums)


###################################### Finding the values from the measurement parameters in the config file 

        values_of_chosen_measurement_parameters = []

        param_list,value_list = get_all_measurement_parameters_and_values()

        for all_params,all_values in zip(param_list,value_list):
            if all_params in df_measurement_params:
                values_of_chosen_measurement_parameters.append(all_values)   


        display_Params(df_measurement_params,values_of_chosen_measurement_parameters)

###################################### Finding the values from the lens parameters in the config file 

#             values_of_chosen_lens_parameters = []        

#             rc_parameters,rc_values = get_rc_parameters_and_values(given_rc_num)

#             for all_rc_params,all_rc_values in zip(rc_parameters,rc_values):
#                 if all_rc_params in df_lens_params:
#                     values_of_chosen_lens_parameters.append(all_rc_values)


#             display_RC_Params(df_lens_params,values_of_chosen_lens_parameters,lens_canvas_chosen)
            
        ##print('df_mtf_values_new',df_mtf_values_new)
        
#         del df_mtf_values_new[-1]

#         wafer_window_limits[toplevel_number] = df_mtf_values_new
    
#         #print('this is the limit :' , wafer_window_limits)
        
#################################### Reload in the pass fail map 

#         if len(toplevel_window_names)>0:
    
#             load_in_wafer_map_and_legend(toplevel_number)
        
        

In [28]:
#### loading in the lens parameters for the wafer map window

def loading_config_wafer_map_window(rc_parameters,rc_values,canvas):
    
    chosen_config = config_file_dropdown.get()
    current_working_directory = os.getcwd()
    Save_file_name = current_working_directory + '/' + chosen_config 

    lens ='[Lens Parameters]'
    measurement = '[Measurement Parameters]'


    with open(Save_file_name,'r') as reader:
        config_list = reader.readlines()
        new_df = pd.DataFrame(config_list)[0].str.rsplit(":",1, expand=True)
        new_df = new_df.replace('\n','', regex=True) 
        new_df.columns = ['Parameters', 'Values']

        mtf_lens_measurement_df = new_df.loc[(new_df['Parameters'].str.startswith(lens)) | (new_df['Parameters'].str.startswith(measurement)) ]  #add in wafer map display here       # only contains the header of each section ([MTF Specifications],[Lens Parameters])
        index_list = mtf_lens_measurement_df.index.tolist()

        df_lens = new_df.iloc[index_list[0]:index_list[1]]

###################################### finds all the lens parameters

        df_lens_params = df_lens['Parameters'].values.tolist()[1:]
        df_lens_params = [x.strip() for x in df_lens_params]                                  # list of params names
        
#         #print('df_lens_params',df_lens_params)

###################################### Finding the values from the lens parameters in the config file 

        values_of_chosen_lens_parameters = []



        for all_rc_params,all_rc_values in zip(rc_parameters,rc_values):
#             #print('all_rc_params::',all_rc_params,'all_rc_values::',all_rc_values)
            if all_rc_params in df_lens_params:
#                 #print('all_rc_params:',all_rc_params,'all_rc_values:',all_rc_values)
                values_of_chosen_lens_parameters.append(all_rc_values) 


        display_RC_Params(df_lens_params,values_of_chosen_lens_parameters,canvas)


In [29]:
def load_in_wafer_map_and_legend(toplevel_number):  # this is used to refresh the wafer map when user changes the config so that we can load in the correct pass fail map 
    
    
#     window_frame = window_canvas.nametowidget("wafer_map_window_frame") 
    
    
    temp_widget = ".!"+toplevel_number  # make the toplevel_number equal to the window number just to check if we got the correct window
    
    #### check for the actual parameters that we have to pass to main_wafermap 
    
    actual_window = [toplevel_window_names[i] for i in range(len(toplevel_window_names)) if temp_widget == str(toplevel_window_names[i])][0]
    actual_window_frame = actual_window.nametowidget('wafer_map_window_container').nametowidget('window_canvas').nametowidget('wafer_map_window_frame')
    actual_window_canvas = actual_window.nametowidget('wafer_map_window_container').nametowidget('window_canvas')
    actual_window_scrollbar = actual_window.nametowidget('wafer_map_window_container').nametowidget('window_scrollbar')
    
#     #print('actual_window',actual_window)
#     #print('actual_window_frame',actual_window_frame)
#     #print('actual_window_canvas',actual_window_canvas)
#     #print('actual_window_scrollbar',actual_window_scrollbar)

    
    ### reset the parameter when the pass fail map is loaded in 
    
    ## getting the data 
    
    wafer_window_parameters[toplevel_number] = "blank"   # reset the wafer parameters to blank 
    
    wafer_frame_parsed = actual_window_frame.nametowidget('datfilename_frame').nametowidget('wafer_frame')
    
    pixelVirtual_parsed = tk.PhotoImage(width=pixelVirtual_width, height=pixelVirtual_height)
    
    tooltip_1_parsed = Pmw.Balloon(wafer_frame_parsed)
    
    ### load in the correct toplevel filepath 
    
#     #print('load_in_wafer_map_and_legend toplevel_number',toplevel_number)
#     
    toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
    rows = [x.rsplit(',')[0] for x in rc_list]
    cols = [x.rsplit(',')[1] for x in rc_list]

    col_max_parsed = int(max(cols, key=float)) 
    row_max_parsed = int(max(rows, key=float))
    col_min_parsed = int(min(cols, key=float))
    row_min_parsed = int(min(rows, key=float))
    
    wafer_map_button_width_parsed,wafer_map_button_height_parsed = wafer_button_sizes(col_max_parsed)
    
    ### refresh the wafer parameters and refresh the wafer map (main_wafermap(refresh the pass fail map))
    
    wafer_param_dropdown_selection_widgets(actual_window,actual_window_frame,actual_window_canvas,90,10,actual_window_scrollbar,wafer_frame_parsed,pixelVirtual_parsed,tooltip_1_parsed,wafer_map_button_width_parsed,wafer_map_button_height_parsed,'blank',col_max_parsed,col_min_parsed,row_min_parsed,row_max_parsed) 
    
    main_wafermap(actual_window,actual_window_frame,actual_window_canvas,actual_window_scrollbar)
    

In [30]:
###### wafer map button sizes

def wafer_button_sizes(col_max_parsed):
    
    ## if max columns > 50 
    
    if col_max_parsed > 50 :
        
        wafer_map_button_width = 8
        
        wafer_map_button_height = 5
        
        return(wafer_map_button_width,wafer_map_button_height)
    
     ## if max columns between 40 & 50 
        
    elif col_max_parsed > 40 and col_max_parsed <= 50 :
        
        wafer_map_button_width = 10
        
        wafer_map_button_height = 7
        
        return(wafer_map_button_width,wafer_map_button_height)
        
    ## if max columns between 30 & 40 
    
    elif col_max_parsed > 30 and col_max_parsed <= 40 :
        
        wafer_map_button_width = 12
        
        wafer_map_button_height = 9
        
        return(wafer_map_button_width,wafer_map_button_height)
    
    ## if max columns between 20 & 30 
        
        
    elif col_max_parsed > 20 and col_max_parsed <= 30 :
        
        wafer_map_button_width = 14
        
        wafer_map_button_height = 11
        
        return(wafer_map_button_width,wafer_map_button_height)
        
    ## if max columns between 10 & 20 
        
    elif col_max_parsed > 10 and col_max_parsed <= 20 :
        
        wafer_map_button_width = 16
        
        wafer_map_button_height = 13
        
        return(wafer_map_button_width,wafer_map_button_height)
    
    ## if max columns less than 10
        
    else:
        
        wafer_map_button_width = 18
        
        wafer_map_button_height = 15
        
        return(wafer_map_button_width,wafer_map_button_height)
        
        
    

In [31]:
######################################## creation of widgets for the input table

def input_table(y_pos,next_y_pos):
    
    global inptvals
    global sag_tan_tfc          # global list of names for the s1 to t9 input boxes
#     global Save_file_entry
    global input_table_labelframe
    
    rows = []
    inptvals = []
    sag_tan_tfc = []                # store all the names that we have 
    
    
    input_table_labelframe = LabelFrame(file_input_frame, text="Settings " , height = input_table_labelframe_height , width = input_table_labelframe_width , name = 'input_table_labelframe')
    input_table_labelframe.place(x =  input_table_labelframe_x_position , y=  input_table_labelframe_y_position)

    
######################################################################################## Sagittal and tangental Labels above the input table

    
    fontStyle = tkFont.Font(family="Lucida Grande", size=11)
    
    sagittal = Label(input_table_labelframe,text = "Sagittal" , font=fontStyle)
    sagittal.place(x = sagittal_label_x_position, y= sagittal_label_y_position )
    
    tangental = Label(input_table_labelframe,text = "Tangental" , font=fontStyle)
    tangental.place(x = tangental_label_x_position , y= tangental_label_y_position )
    
    
######################################################################################### loop to make the table   


    Sagittal = "s"
    Tangental = "t"

    for i in label_list_input_table:   

        params = Label(input_table_labelframe,text = i + " :")    # making the labels in the settings labelframe
        params.place(x = params_x_position , y=  y_pos + 25 )
        
        # forming the name of the S1 to S9 
        
        S_num = i.rsplit(" ",1)[1]                                                              # get the number from the back of the component labels
        S_pos = Sagittal + str(S_num)
        sag_tan_tfc.append(S_pos)
        
        # forming the name of the T2 to T9
        
        T_num = int(i.rsplit(" ",1)[1]) 
        T_pos = Tangental + str(T_num)
        sag_tan_tfc.append(T_pos)

        
        for num_cols in range(1):
            s = Entry(input_table_labelframe,relief=RIDGE,bd = s_bd , name = S_pos)                    # Name each box with a new S_positon (S1 to S9)
            s.place(x = s_x_position, y = y_pos + 25)
            s.insert(0, 0)

            y_pos += y_padding 
        
        for num_cols in range(1):
            if i != label_list_input_table[0]:                                                  # first camera in the list of camera names
                t = Entry(input_table_labelframe,relief=RIDGE,bd = t_bd , name = T_pos)                # Name each box with a new T_positon (T2 to T9)
                t.place(x = t_x_position, y = next_y_pos + 25)
                t.insert(0, 0)

                next_y_pos += y_padding 
            else:
                continue
                
    sag_tan_tfc.pop(1)                                                                          # remove the value t1

###################################################################################### TFC box 
    
    
    sag_tan_tfc.append(tfc_widget_name)
    
    TFC_label = Label(input_table_labelframe,text = "TFC :")
    TFC_label.place(x = TFC_label_x_position, y= y_pos + 30)
    
    TFC = Entry(input_table_labelframe, relief=RIDGE,bd = TFC_entry_bd, name = tfc_widget_name)
    TFC.place(x = TFC_entry_x_position,y= y_pos + 30)
    TFC.insert(0, 0)
    


In [32]:
############################### check for errors in the user input

def check_input(sag_tan_tfc):
    
    highlight_COLOR = "red"
    
    inptvals.clear()  # keeps clearing the list until the final time the button has been clicked ( so basically the inputvalues will not be placed into the config file unless all values in the s1 to t9 inputs are correct)
    count_wrong = 0   # counts number of errors users made while inputting variables
    
    ######## loop through widget names 
    
    for i in sag_tan_tfc:
        sag_tan_val = input_table_labelframe.nametowidget(i).get()
#         ##print('type(sag_tan_val)',type(sag_tan_val),sag_tan_val)
        
        ###### allow TFC to be able to load in a float (as numbers need to be below 1 )
    
        if i == 'tfc':
#             ##print('sag_tan_val',sag_tan_val)
            try:
                sag_tan_val = float(sag_tan_val)
            
            except ValueError:
                
                input_table_labelframe.nametowidget(i).delete(0, END)
                input_table_labelframe.nametowidget(i).config(highlightthickness=1, highlightbackground= highlight_COLOR)
                
                raise Exception(messagebox.showwarning("showwarning", "Warning ! Please enter a valid number in the highlighted cells !"))     # raise exception 
                
        
            check_float = isinstance(sag_tan_val, float)
            
            if check_float == True:
                if sag_tan_val < 0 :                  # 0 to (lower_limit_tfc_value)
                    count_wrong += 1
                    input_table_labelframe.nametowidget(i).delete(0, END)
                    input_table_labelframe.nametowidget(i).config(highlightthickness=1, highlightbackground= highlight_COLOR)
                else:
                    inptvals.append(sag_tan_val)
                    input_table_labelframe.nametowidget(i).config(highlightthickness=0)
        
        
        ########## check if the value keyed in is a number 
        
        elif sag_tan_val.isdigit():
            sag_tan_val = int(sag_tan_val)

            if sag_tan_val > 100 or sag_tan_val < 0:    # 100 to (upper_limit_s1_to_t9_values) , 0 to (lower_limit_s1_to_t9_values)
                count_wrong += 1
                input_table_labelframe.nametowidget(i).delete(0, END)
                input_table_labelframe.nametowidget(i).config(highlightthickness=1, highlightbackground= highlight_COLOR)
            else:
                inptvals.append(sag_tan_val)
                input_table_labelframe.nametowidget(i).config(highlightthickness=0)
        
        ######## check if keyed in value is empty (no values entered )  
        
        ## if no value entered intialize the value as 0 

        elif sag_tan_val == '':
            sag_tan_val = 0
            inptvals.append(sag_tan_val)
            input_table_labelframe.nametowidget(i).config(highlightthickness=0)

        else:
            input_table_labelframe.nametowidget(i).delete(0, END)
            input_table_labelframe.nametowidget(i).config(highlightthickness=1, highlightbackground= highlight_COLOR)
            count_wrong += 1
            input_table_labelframe.nametowidget(i).delete(0, END)

            
    ############# check the number of errors that user has made  (had to use exception to stop the process of returning the s1 to tfc values)
    
    if count_wrong > 0 :
        raise Exception(messagebox.showwarning("showwarning", "Warning ! " + str(count_wrong) + " count(s) of Error, Please input in a number that is between 0 and 100 in the highlighted cells !"))     # raise exception 
        
    
    return(inptvals)

In [33]:
############################################################### Update settings button function

def savedata_overwrite(cwd,sag_tan_tfc):
    
    
    chosen_config = config_file_dropdown.get()
    Save_file_name = cwd + '/' + chosen_config   # CHANGE TO GET FROM THE DROPDOWNBOX
    
    if len(chosen_config) > 0:
    
        check_input(sag_tan_tfc)
    #################################################################################################### Checking for if the filepath exists and if it exists we will ask the users if they want to overwrite , else we will just write to a new file 

        if os.path.exists(Save_file_name):
            inptvals.insert(0,None)           # insert none into inputvals as the first variable


            answer = messagebox.askquestion("askyesno", "Warning ! Would you like to Update this Config file ? ")  # answer for either yes or no to whether user wants to refresh the config

    #################################################################################################### make the text file to be replaced a dataframe , then remove the data that the user does not want from that dataframe and re write into the textfile again
            if answer == 'yes' :

                with open(Save_file_name,'r') as file_reader:     
                    
                    #### load the entire text file into a dataframe & replace the '\n' & name the columns Parameters and Values

                    file_config_list = file_reader.readlines()
                    new_df = pd.DataFrame(file_config_list)[0].str.rsplit(":",1, expand=True)
                    new_df = new_df.replace('\n','', regex=True) 
                    new_df.columns = ['Parameters', 'Values']
                    
                    ##### get a dataframe of just the headers [mtf_specs,lens and measurement]

                    mtf_lens_measurement_df = new_df.loc[(new_df['Parameters'].str.startswith(mtf_specs)) | (new_df['Parameters'].str.startswith(lens)) | (new_df['Parameters'].str.startswith(measurement))]
                    index_list = mtf_lens_measurement_df.index.tolist()
                    
                    ##### gets just the s1 to t9 

                    df_mtf = new_df.iloc[index_list[0]:index_list[1]]                                   # create a dataframe for just the mtf numbers 
                    df_lens = new_df.iloc[index_list[1]:index_list[2]]
                    df_measurement = new_df.iloc[index_list[2]:]

                    ##### gets just lens parameters & measurement parameters
                    
                    df_lens_params = df_lens['Parameters'].values.tolist()
                    df_lens_params = [x.strip() for x in df_lens_params]
                    df_measurement_params = df_measurement['Parameters'].values.tolist()
                    df_measurement_params = [x.strip() for x in df_measurement_params]

                    df_mtf['Parameters'] = df_mtf['Parameters'].astype(str) + ':' 

                    df_mtf['Values'] = inptvals                                                        # enters the values columns and just places the inptvals from the check_input
                    
                    ##### only write in the undeleted lens and measurement parameters

                    if len(rc_deleted_params) > 0:
                        for rc_params in rc_deleted_params:
                            if rc_params in df_lens_params:
                                df_lens_params.remove(rc_params)


                    if len(dat_deleted_params)>0:
                        for dat_params in dat_deleted_params:
                            if dat_params in df_measurement_params:
                                df_measurement_params.remove(dat_params)
                                
                    ###### remake the 3 dataframes 

                    df_final_lens = pd.DataFrame(df_lens_params)
                    df_final_lens["Values"] = ""                                                      # ensure that values column is blank for the lens parameters
                    df_final_lens.columns = ['Parameters','Values']
                    df_final_measurement = pd.DataFrame(df_measurement_params)                        # ensure that values column is blank for the measurement parameters
                    df_final_measurement["Values"] = ""
                    df_final_measurement.columns = ['Parameters','Values']

                ####### concatenate the 3 dataframes together and rewrite the text file
                
                
                last_df = pd.concat([df_mtf,df_final_lens,df_final_measurement],ignore_index=True)
                last_df.fillna("",inplace=True)

                numpy_array = last_df.to_numpy()
                np.savetxt(Save_file_name, numpy_array, fmt = "%s")


                load_in_selected_config_file(chosen_config,Save_file_name)  # refresh config again

            else:
                pass
    
    else:
        
        messagebox.showwarning("showwarning", "Warning ! Please select a Config file to Update !")

In [34]:
####################################################  saves the new config file / overwrite old config file for 1st time users through the popup file 

def get_filename_popup(save_file_name,config_popup):
    
#     ##print('this is the given_rc_num' , given_rc_num)
    
    chosen_config_filename = save_file_name.get()
    current_wd = os.getcwd()
    filetype = config_file_type.split('*')[1]
    config_file_name = current_wd + '/' + chosen_config_filename + filetype
    
    param_list,value_list = get_all_measurement_parameters_and_values()
    
    rc_parameters,rc_values = get_rc_parameters_and_values(get_global_Rc_num)   # purposely set a random variable  (users will just get all the rc parameters)
    
    with open(config_file_name,'w') as savefile:
        
        savefile.write(mtf_specs)
        savefile.write('\n')

        for value ,sag_tans in zip(inptvals,sag_tan_tfc):
            savefile.write(sag_tans + ":" + str(value))
            savefile.write('\n')

        savefile.write(lens)
        savefile.write('\n')
        
        #### write the rc parameters (Users will just have all the rc data (no data will be taken away))
        
        for rc_params in rc_parameters:
            if rc_params not in rc_deleted_params:
                savefile.write(rc_params)
                savefile.write('\n')

        savefile.write(measurement)
        savefile.write('\n')
        

        for dat_params in param_list:                                       # loop through all the parameters if user has deleted any items from the parameters listbox , then the deleted parameters will not be written into the text file 
            if dat_params not in dat_deleted_params:
                savefile.write(dat_params)
                savefile.write('\n')
        
    config_dropdown(chosen_config_filename)  # refresh the config dropdown after creating a new config file
        
    config_popup.destroy()

    mainloop()

# Wafer map v2 (gradient)    [GRID & PLACE TOGETHER WILL BE FINE]

In [35]:
def mainplot1(given_x,given_y,labels):
    
    if labels == 'S1' or labels == 'T1':
        trace = go.Scatter(
        x=given_x,
        y = given_y,
        mode='lines',
        line_shape='spline',
        name = labels,
        marker=dict(
            color = 'rgb(31,73,125)'
        )
        )
    if labels == 'S2' or labels == 'S3' or labels == 'S4' or labels == 'S5':
        trace = go.Scatter(
        x=given_x,
        y = given_y,
        mode='lines',
        line_shape='spline',
        name = labels,
        marker=dict(
            color = 'rgb(0,176,80)'
        )
        )
    if labels == 'S6' or labels == 'S7'or labels == 'S8'or labels == 'S9':
        trace = go.Scatter(
        x= given_x,
        y = given_y,
        mode='lines',
        line_shape='spline',
        name = labels,
        marker=dict(
            color = 'rgb(246,142,56)'
        )
        )
    if labels == 'T2' or labels == 'T3' or labels == 'T4' or labels == 'T5':
        trace = go.Scatter(
        x=given_x,
        y = given_y,
        mode='lines',
        line_shape='spline',
        line={'dash': 'dash'},
        name = labels,
        marker=dict(
            color = 'rgb(0,176,80)'
        )
        )
    if labels == 'T6' or labels == 'T7'or labels == 'T8'or labels == 'T9':
        trace = go.Scatter(
        x= given_x,
        y = given_y,
        mode='lines',
        line_shape='spline',
        line={'dash': 'dash'},
        name = labels,
        marker=dict(
            color = 'rgb(246,142,56)'
        )
        )
    
    return(trace)


In [36]:
############################# Draw the plotly graph

def creategraph1(given_list,ffl_num,frame,r_value,c_value,given_tfc_list):
    
    fig = go.Figure()            # clear the previous graph everytime a new one is created
    data = given_list
    
    
    
    layout = go.Layout(
      width=graph_width,
      height=graph_height,
      title = "MTF Curve " + r_value + "," + c_value,
      xaxis_title="Z-Height, MM",
      yaxis_title="MTF, %",
      margin=go.layout.Margin(
            l=graph_layout_margin_left, #left margin
            r=graph_layout_margin_right, #right margin
            b=graph_layout_margin_bottom, #bottom margin
            t=graph_layout_margin_top  #top margin
        )
    )
    fig = go.Figure(data,layout,layout_yaxis_range=[0,100])
    fig.update_yaxes(dtick=10)
    fig.update_xaxes(dtick=0.01)
    
    ################################# Plot the FFL number line 
    
    if ffl_num != 0 :
        
        fig = fig.add_vline(x=ffl_num, line_width= ffl_number_line_width , line_dash="dash", line_color= ffl_line_color)   # add a vertical line 
        
        rounded_ffl_num = round(ffl_num, 3)
        
        string_ffl_num = str(rounded_ffl_num)

        fig = fig.add_annotation(x=ffl_num,y = ffl_number_annotation_y_position , text= "   FFL :" + "   " + string_ffl_num)  # add a annotation onto the vertical line
    
    ################################ Plot the TFC line
    
    plot_tfc_line(given_tfc_list,fig)
    
    
    ############################### convert the graph to an image then resize the image 

    img_bytes = fig.to_image(format="png")

    img = PIL.Image.open(io.BytesIO(img_bytes))                 # graph as an image
    
    img = img.resize((graph_image_width,graph_image_height))    # resize the image 
    
    ph = ImageTk.PhotoImage(img)
    
    label = Label(frame, image=ph)
    label.place(x = graph_x_position,y = graph_y_position)      # placing the graph image on the wafer map
    label.image=ph  #need to keep the reference of your image to avoid garbage collection
    
    return fig

In [37]:
def plot_tfc_line(given_tfc_list,fig):
    
    if given_tfc_list != 'LENS FAIL':   # change to len(given_tfc_list) >0

        ##print(given_tfc_list)
        
#         try:   # if more than 1 tfc    (used try and except as we cannot determine the numbers of TFCs it could be just 1 list or multiple lists in a list)
            
        lower_upper_boundary_change = 0

        for tfcs in range(len(given_tfc_list)):

            for tfc in given_tfc_list[tfcs]:

                fig = fig.add_vline(x=tfc, line_width=1, line_dash="dash", line_color = tfc_line_color)

                rounded_tfc = round(tfc, 2)

                string_tfc_num = str(rounded_tfc)     

                if (lower_upper_boundary_change % 2) == 0:

                    annotation_text = "LB: "

                    annotation_tfc_pos = tfc + 0.005

                    fig = fig.add_annotation(x=annotation_tfc_pos,y = tfc_annotation_y_position , text= annotation_text + string_tfc_num  , textangle= tfc_textangle )

                else: 

                    annotation_text = "UB: "

                    fig = fig.add_annotation(x=tfc,y = tfc_annotation_y_position , text= annotation_text + string_tfc_num  , textangle= tfc_textangle )


                lower_upper_boundary_change += 1

    else:
        
        pass
    

In [38]:
#################### Main graph function

def display_graph1(final_csv_df,ffl_number,frame,r_value,c_value,given_tfc_list):
    
#     global x
#     global y
#     global s1_t9_df_cols
    
    try:
        ffl_number = float(ffl_number)
    except ValueError:
        ffl_number = 0
        
    trace_lists = []
    
    s1_t9_df =  final_csv_df.drop(columns=['Position','No. of Scans','R','C','Z distance'])
    s1_t9_df_cols = s1_t9_df.columns
    x = final_csv_df['Z distance'].values
    for i in range(len(s1_t9_df_cols)):
        y = s1_t9_df[[x for x in s1_t9_df_cols][i]].values
        trace = mainplot1(x,y,[x for x in s1_t9_df_cols][i])
        trace_lists.append(trace)

    graph_returned = creategraph1(trace_lists,ffl_number,frame,r_value,c_value,given_tfc_list)
    
#     return graph_returned

In [39]:
############### Create empty graph if the csv data is empty / there is a valueerror

def create_empty_graph(given_list,frame,r_value,c_value):
    
    fig = go.Figure()
    data = given_list
    layout = go.Layout(
      width=graph_width,
      height=graph_height,
      title = "MTF Curve " + r_value + "," + c_value,
      xaxis_title="Z-Height, MM",
      yaxis_title="MTF, %",
      margin=go.layout.Margin(
            l=graph_layout_margin_left, #left margin
            r=graph_layout_margin_right, #right margin
            b=graph_layout_margin_bottom, #bottom margin
            t=graph_layout_margin_top  #top margin
        )
    )
    fig = go.Figure(data,layout,layout_yaxis_range=[0,100])
    
    fig = fig.add_annotation(x=empty_graph_annotation_x_position, y=empty_graph_annotation_y_position,
            text="Through Focus data not available for this RC number!",
            showarrow = False)

    img_bytes = fig.to_image(format="png")

    img = PIL.Image.open(io.BytesIO(img_bytes))                 # graph as an image
    
    img = img.resize((graph_image_width,graph_image_height))
    
    ph = ImageTk.PhotoImage(img)
    
    
    label = Label(frame, image=ph)
    label.place(x = empty_graph_x_position,y = empty_graph_y_position)
    label.image=ph  #need to keep the reference of your image to avoid garbage collection
    
    return fig

In [40]:
############ Function to create the empty graph

def empty_graph(frame,r_value,c_value):

    trace_lists = []
    
    graph_returned = create_empty_graph(trace_lists,frame,r_value,c_value)
    
#     return graph_returned

In [41]:
############ Finding the range of the colors (lower & upper percentile)

def finding_range(ffl_list,given_lower_percentile,given_upper_percentile):  # parse in percentile number

    array_of_ffl = np.array(ffl_list)

    percentile_1 = np.percentile(array_of_ffl, given_lower_percentile)       # returns the lower percentile
    
    percentile_2 = np.percentile(array_of_ffl, given_upper_percentile)       # returns the upper percentile
    
    range_ffl = percentile_2 - percentile_1  # if this is o then its going to fail

    return range_ffl,percentile_2,percentile_1

In [42]:
def getcol(ffl,given_lower_percentile,given_upper_percentile,given_ffl_list):                       # get the gradient color
    
    temp_ffl = ffl
    
    green = Color("green")             # set start color to green
    
    range_ffl,upper_percentile,lower_percentile = finding_range(given_ffl_list,given_lower_percentile,given_upper_percentile)    # given ffl list can be any other parameter 
    
    colors = list(green.range_to(Color("red"),100))                          # changes from green to red and splits it into 100 numbers 
    
    
    ##### check if the upper-lower percentile is 0 
    
    if range_ffl == 0:
        
        if temp_ffl > upper_percentile :
            
            col_index = 100  
            
        elif temp_ffl < lower_percentile:
            
            col_index = 0
        
        else:
            
            color = 'yellow'
            
            return color
            
        
    else:
    
    
        if temp_ffl > upper_percentile :   # to make numbers below 90th percentile have the same color as the 90th percentile

            temp_ffl = upper_percentile 


        if temp_ffl < lower_percentile:    # to make numbers above the 10th percentile have the same color as the 10th percentile

            temp_ffl = lower_percentile


        finding_1_percent = range_ffl/100   # find the value at 1 percent 

        col_index = round(((temp_ffl-lower_percentile)/finding_1_percent))     

    
    if col_index == 0 :

        return(colors[col_index])
    
    else:
        
        
        col_index = col_index - 1

        return(colors[col_index])                # returns the specific color 
    

In [43]:
def get_rc_parameters_and_values(given_rc_num):
    
    ########## used mainly to create a new config , if the user wants to create a new config we will just take the first rc number in the rc list of that DAT file to generate a list of parameters (but since the lens parameters are on the new window \n
    #########  we need to use this way to first populate the config with just the base lens parameters , which afterwards they can then update the settings in the config file )
    
    
    try:
#     #print()
    
        toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]

        new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
        
        final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,toplevel_number)

        rc_header_df = new_final_df_final_df

        rc_parameters = rc_header_df.columns.tolist()

        ############### if user has not changed the DAT file and opened a new wafer map

        
        last_scan_dictionary ={i:rc_list.count(i) for i in rc_list}
        
        final_scan_num = last_scan_dictionary[given_rc_num]
#         print('final_scan_num',final_scan_num)
        
        rc_values =  new_final_df_final_df.loc[(new_final_df_final_df['Position'] == given_rc_num)].loc[(new_final_df_final_df['No. of Scans'] == final_scan_num)].iloc[0].values.tolist()
        
#         print('rc_parameters',rc_parameters,len(rc_parameters))
#         print('rc_values',rc_values,len(rc_values))

        del rc_parameters[2:6]
        del rc_parameters[-3]
        del rc_parameters[-3]



        del rc_values[2:6]
        del rc_values[-3]
        del rc_values[-3]


        ######### add the S1 to T9
        
#         print('steps_list',s1_t9_list)
        
        rc_parameters.extend(s1_t9_list)
        rc_values.extend(new_final_df_camPosDf.iloc[:,-1].where(new_final_df_camPosDf['Position'] == given_rc_num).dropna().tolist()[:len(s1_t9_list)])
        

        return (rc_parameters,rc_values)
        
            
#     ############### to ensure that the user can still create a new config if the toplevel_number doesnt exist or the toplevel_number is not defined 
    
    except (KeyError,NameError,IndexError):
        
        print('exception keywrror')
        
        rc_header_df = new_final_df.final_df

        rc_parameters = rc_header_df.columns.tolist()
        
        last_scan_dictionary ={i:new_final_df.rc.count(i) for i in new_final_df.rc}
        
        final_scan_num = last_scan_dictionary[given_rc_num]

        rc_values =  new_final_df.final_df.loc[(new_final_df.final_df['Position'] == given_rc_num)].loc[(new_final_df.final_df['No. of Scans'] == final_scan_num)].iloc[0].values.tolist()


        del rc_parameters[2:6]
        del rc_parameters[-3]
        del rc_parameters[-3]



        del rc_values[2:6]
        del rc_values[-3]
        del rc_values[-3]


        ######### add the S1 to T9
        
        FILENAME_csv = filepath.rsplit('/',1)[1][:-4] + '.csv'                                   # To get the csv filename
        
        path = filepath.rsplit('/',1)[0]
        
        csv_filepath = os.path.join(path,FILENAME_csv)
        
        csv_data = csv(csv_filepath,new_final_df.final_df,'0')
        step_list = csv_data.camComp  # list of steps s1-t9

        rc_parameters.extend(step_list)
        rc_values.extend(new_final_df.camPosDf.iloc[:,-1].where(new_final_df.camPosDf['Position'] == given_rc_num).dropna().tolist()[:len(step_list)])


        return (rc_parameters,rc_values)
        
    
    

In [53]:
def getgraph(rc_select,window,frame,canvas,upper_percentile,lower_percentile,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max):
    
#     ##print('this is the getgraph upper_percentile', upper_percentile)
#     ##print('this is the getgraph lower_percentile', lower_percentile)
    
    toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
#     global counter_clicked
    
#     global rc_num
    
#     global lens_canvas_chosen
    
    rc_select = str(rc_select)
    
    rc_num = rc_select.split(".")[-1]
    
    
    get_root_window_name = rc_select.split('.!')[1].split('.')[0]    # gets the window name of the selected valuye (finds out which window did the user click the rc on)
    
    
    r_value = rc_num.split(',')[0]
    c_value = rc_num.split(',')[1]
     
    
    ffl_number = new_final_df_final_df['FFL'].where(new_final_df_final_df['Position'] == rc_num).dropna().values.tolist()[0]   # getting the ffl of the clicked rc number 
    
    lens_parameters,lens_values = get_rc_parameters_and_values(rc_num)
    
    config_chosen = config_file_dropdown.get()    # getting the config file info 
    
    
    ######################### Check if the user has clicked onto the wafer map , if the user has not clicked onto the wafer map do not allow users to change the config file
    
    if window not in toplevel_window_names :
        
        toplevel_window_names.append(window)
        
    
    ######################### To make sure the correct wafer parameters are loaded in for the correct window
    
    if get_root_window_name not in wafer_window_parameters:  ## if user has not selected a wafer parameter(if user has not changed color)
        
        selected_wafer_parameter = "blank"
        
        #print('selected_wafer_parameter::',selected_wafer_parameter,'get_root_window_name',get_root_window_name)
        
    else:
        
        selected_wafer_parameter = wafer_window_parameters[get_root_window_name]
        
        #print('selected_wafer_parameter',selected_wafer_parameter,'get_root_window_name',get_root_window_name)
        
    ######################### Make sure that correct config file is loaded in for the correct wafer map window  (if config is present, load in , else just have base value)

    if len(config_chosen) == 0:

        display_RC_Params(lens_parameters,lens_values,canvas)  

#         ##print('selected_wafer_parameter ::',selected_wafer_parameter)

        wafer_param_dropdown_selection_widgets(window,frame,canvas,upper_percentile,lower_percentile,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,selected_wafer_parameter,col_max,col_min,row_min,row_max)
    
    
    else:
        
        ####### check if user selected to create a new config file , but closed the config file popup window before entering a config filename

        if config_chosen == 'Create a new config file':

            config_file_dropdown.set('')
            
            display_RC_Params(lens_parameters,lens_values,canvas)  

            wafer_param_dropdown_selection_widgets(window,frame,canvas,upper_percentile,lower_percentile,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,selected_wafer_parameter,col_max,col_min,row_min,row_max)
        
        else:
        
            wafer_param_dropdown_selection_widgets(window,frame,canvas,upper_percentile,lower_percentile,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,selected_wafer_parameter,col_max,col_min,row_min,row_max)

            loading_config_wafer_map_window(lens_parameters,lens_values,canvas)
            
#     #print(wafer_window_configs)
    ##################### Mtf limits , ensure that the user will have the same mtf limits on the same window 
    
    TFC,MTF_Lim = mtf_limits_same_window(get_root_window_name)
    
    try:
        
        toplevel_specific_filepath = wafer_window_subfolder_name[toplevel_number]
        
        final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,toplevel_number)
        
        last_scan_dictionary ={i:rc_list.count(i) for i in rc_list}
        
        
        final_scan = last_scan_dictionary[rc_num]
#         print('final_scan',final_scan)
        
        split_by_rc_dataframe = final_csv_df.where((final_csv_df['Position'] == rc_num) & (final_csv_df['No. of Scans'] == final_scan)).dropna()   # we wil only get the parts of the dataframe where the position is equal to the rc number user selected
        
#         #print('split_by_rc_dataframe',split_by_rc_dataframe)
        #print('MTF_Lim',MTF_Lim,len(MTF_Lim))
        
        tfc_list = getTFC(split_by_rc_dataframe,MTF_Lim)  # getting the tfc list

        display_graph1(split_by_rc_dataframe,ffl_number,canvas,r_value,c_value,tfc_list)

    except ValueError:
        
#         ##print('error')
        
        empty_graph(canvas,r_value,c_value)
    
        

        

def main_wafermap(window , frame ,canvas,window_scrollbar):
    
#     ##print(type(window))
#     global datfilename_frame
    
    window_scrollbar.pack(side = RIGHT , fill = Y)
    
    frame.bind('<Enter>',lambda event: _bound_to_mousewheel(event,canvas))
    frame.bind('<Leave>',lambda event: _unbound_to_mousewheel(event,canvas))
    
    Pmw.initialise(window)                               # for the tooltip popup (need to have a widget as the input )
    
    ######## get the correct window number
    
    window_num = str(window)
    window_number = window_num.split(".!")[-1]
    
    toplevel_specific_filepath = wafer_window_subfolder_name[window_number]
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
    rows = [x.rsplit(',')[0] for x in rc_list]
    cols = [x.rsplit(',')[1] for x in rc_list]

    col_max = int(max(cols, key=float)) 
    row_max = int(max(rows, key=float))
    col_min = int(min(cols, key=float))
    row_min = int(min(rows, key=float))

    pixelVirtual = tk.PhotoImage(width=pixelVirtual_width, height=pixelVirtual_height)                 # make the window units in pixels instead of inches  (need to initalize compound = 'c' in the button initialization to place text inside the button)
    
    
    datfilename_frame = Frame(frame,width = width_of_frame , height= height_of_frame,name = 'datfilename_frame')          # makes the frame another frame to place the filenames 
    wafer_frame = Frame(datfilename_frame,width = width_of_frame , height= height_of_frame,name = 'wafer_frame')    # makes the wafer_frame a frame of 

    tooltip_1 = Pmw.Balloon(wafer_frame)       # create tooltip 
#     #print('this is tooltip_1',tooltip_1)
    

    ######### display the filename at the top of the wafer map
    
    datfilename_font = tkFont.Font(family='Helvetica',size=14)
    
    get_correct_frequency = frequency_csv_for_each_Window[window_number]
    
    frequency_number_to_display = get_correct_frequency.rsplit('_',1)[1].split('.')[0]
    
    filename = toplevel_specific_filepath.split('-',1)[0] + ' - ' + frequency_number_to_display        # get only the parts up till no filter (eg:MAGGIE_FUAXA21B00016_no filter)
    print('filename1',filename)
    
    Dat_filename = Label(datfilename_frame,text = filename,font = datfilename_font)
    Dat_filename.place(x = Dat_filename_x_position+35 , y = Dat_filename_y_position)
    
    ######### size of the wafer map buttons
    

    
    font = tkFont.Font(family="Helvetica", size=5, weight="normal")
#     m_len = font.measure("m")

    wafer_map_button_width,wafer_map_button_height = wafer_button_sizes(col_max)
    
    
    ################################### calling the Wafer map legend
    
    wafer_map_legend(col_max,col_min,row_min,row_max,wafer_frame,pixelVirtual,wafer_map_button_width,wafer_map_button_height)
            
    
    ############################################################### Making wafer map
    
    ##### if heat map is selected display the correct wafer map
    
    if file_input_frame.nametowidget('select_frequency_labelframe').nametowidget('heat_map_dropdown').get() == 'Yes':
        
        filename = toplevel_specific_filepath.split('-',1)[0] + ' - Heat Map'
        
        Dat_filename = Label(datfilename_frame,text = filename,font = datfilename_font)
        Dat_filename.place(x = Dat_filename_x_position+35 , y = Dat_filename_y_position)
        
        file_input_frame.nametowidget('select_frequency_labelframe').nametowidget('heat_map_dropdown').set('')
        
        wafer_map_creation([],wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,window , frame ,canvas,window_scrollbar,90,10,col_max,col_min,row_min,row_max,'Yes')
        
    else:
    
        wafer_map_creation([],wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,window , frame ,canvas,window_scrollbar,90,10,col_max,col_min,row_min,row_max,'No')
    
    
    canvas.pack(fill='both', expand=1)
    datfilename_frame.place(x = datfilename_frame_x_position,y = datfilename_frame_y_position)
    wafer_frame.place(x = wafer_frame_x_position,y = wafer_frame_y_position)
    
    
    window.update()
    window.update_idletasks()


    

    window.mainloop()

In [45]:
def mtf_limits_same_window(get_root_window_name):
    
    config_chosen = config_file_dropdown.get()

    toplevel_specific_filepath = wafer_window_subfolder_name[get_root_window_name]
    
    final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,get_root_window_name)
    
#     final_csv_df,PMO,s1_t9_list,steps_list = get_csv_data(get_root_window_name)
    
    
#     if len(config_chosen) == 0 or (toplevel_number not in wafer_window_configs):
        
    MTF_Lim_with_tfc = check_input(sag_tan_tfc)

    MTF_Lim_s1_t9 = [int(MTF_Lim_with_tfc[x]) for x in range(len(MTF_Lim_with_tfc)) if x+1 < len(MTF_Lim_with_tfc)]  # without tfc (17 vals)

    try:
        #print('Try MTF_Lim_with_tfc[-1]',MTF_Lim_with_tfc[-1])
        TFC = float(MTF_Lim_with_tfc[-1])
        #print('Try TFC',TFC)

    except ValueError:

        #print('MTF_Lim_with_tfc[-1]',MTF_Lim_with_tfc[-1])
        TFC = int(MTF_Lim_with_tfc[-1])
        #print('except TFC',TFC)

#         del MTF_Lim[-1]   ## delete the TFC value

    ######## place in the T1
    #print('second final MTF_Lim_s1_t9',MTF_Lim_s1_t9,len(MTF_Lim_s1_t9))

    if len(MTF_Lim_s1_t9) < len(s1_t9_list):     
        #print('MTF_Lim_with_tfc[0]',MTF_Lim_with_tfc[0],type(MTF_Lim_with_tfc[0]))
        MTF_Lim_s1_t9.insert(1, int(MTF_Lim_with_tfc[0])) 

    #print('final MTF_Lim',MTF_Lim_s1_t9,len(MTF_Lim_s1_t9))

    return (TFC,MTF_Lim_s1_t9)
        
#         del MTF_Lim[-1]   ## delete the TFC value  (not sure if need)
        
        ##print('no config MTF_Lim',MTF_Lim)
        
        
#     else:
        
#         temp_MTF_Lim = wafer_window_limits[get_root_window_name]
        
#         try:
#             MTF_Lim = [int(mtf) for mtf in temp_MTF_Lim]   # if the tfc value is a float there will be an error thats why try and except
            
#         except ValueError:
            
#             MTF_Lim = [int(temp_MTF_Lim[x]) for x in range(len(temp_MTF_Lim)) if x+1 < len(temp_MTF_Lim)]
            
#             if len(temp_MTF_Lim) <= len(s1_t9_list):    
# #                 #print('temp_MTF_Lim[0]',temp_MTF_Lim[0],type(temp_MTF_Lim[0]))
#                 MTF_Lim.insert(1, int(temp_MTF_Lim[0]))
            
#             #print('mtf_limits_same_window MTF_Lim',MTF_Lim,len(MTF_Lim))
        
#             tfc_value = float(temp_MTF_Lim[-1])
            
        
#     return MTF_Lim

In [46]:
def wafer_map_creation(given_list_of_users_selected_params,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,window , frame ,canvas,window_scrollbar,given_upper_limit_entered,given_lower_limit_entered,col_max,col_min,row_min,row_max,heat_map_check):
    
    
    ######## search for the correct wafer window 
    
    
    
    window_num = str(window)
    window_number = window_num.split(".!")[-1]
    
    print('window_number::',window_number)
    toplevel_specific_filepath = wafer_window_subfolder_name[window_number]
    print('wafer_map_creation toplevel_specific_filepath',toplevel_specific_filepath)
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)
    
    
    
    try:
        
        
        ############# for the color changes
        
        if len(given_list_of_users_selected_params) > 0:                   # for the changed wafer map  (with color due to the user choosing the wafer map parameters)

            if given_upper_limit_entered != 90 or given_lower_limit_entered != 10:

                for pos , list_vals in zip(rc_list,given_list_of_users_selected_params):
                    row = pos.split(',')[0]
                    col = pos.split(',')[1]


                    try:

                        color = getcol(list_vals,given_lower_limit_entered,given_upper_limit_entered,given_list_of_users_selected_params) 

                    except ValueError:
            #             ffl_number = 999
                        color = 'black'

                    rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )
    #                 ##print('rc_select',rc_select)

                    rc_select.grid(row=row, column=col)

    #                 ##print('these are -the given upper limits',given_upper_limit_entered )
    #                 ##print('these are -the given lower limits',given_lower_limit_entered )

                    rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,given_upper_limit_entered,given_lower_limit_entered,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                    tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it

            else:

                for pos , list_vals in zip(rc_list,given_list_of_users_selected_params):
                    row = pos.split(',')[0]
                    col = pos.split(',')[1]

                    color = getcol(list_vals,given_lower_limit_entered,given_upper_limit_entered,given_list_of_users_selected_params) 

                    rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )

                    rc_select.grid(row=row, column=col)

                    rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                    tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it
                
        ################# for the heat_map
        
        elif heat_map_check == 'Yes':
            
            ### call function
            
            
            color_list = find_heat_map_vals()
#             #print('color_list',color_list)
            
            rcs_in_color_list = [i[0] for i in color_list]
            
            ################ make the heat map
        
            for pos in rc_list:
                row = pos.split(',')[0]
                col = pos.split(',')[1]
                
                if pos in rcs_in_color_list:
                    
#                     #print('pos',pos)
                    
                    position_of_rc = rcs_in_color_list.index(pos)   # finds which index the rc number is in the list_of_tfcs
                    color = color_list[position_of_rc][1]
                

                rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )

                rc_select.grid(row=row, column=col)

#                 rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it

                
        ################# for the pass fail map 

        else:                                             # for the base wafer map (no color )
            
            string_Wafer_frame = str(wafer_frame)
            
            correct_window_name = string_Wafer_frame.split('.!')[1].split('.')[0]
            print('correct_window_name',correct_window_name)
            
            tfc_val,correct_mtf_limits = mtf_limits_same_window(correct_window_name)

            list_of_tfcs = get_list_of_tfcs(tfc_val,correct_mtf_limits,correct_window_name)   # should be same number of elements as new_final_df.rc
        
#             print('list_of_tfcs',list_of_tfcs)
            
            rcs_in_list_of_tfcs = [i[0] for i in list_of_tfcs]

            for pos in rc_list:
                row = pos.split(',')[0]
                col = pos.split(',')[1]

                font = tkFont.Font(family="Helvetica", size=7, weight="normal")
                
#                 print('rcs_in_list_of_tfcs',rcs_in_list_of_tfcs)

                if pos in rcs_in_list_of_tfcs:
                    
#                     print('rcs_in_list_of_tfcs',rcs_in_list_of_tfcs)

                    position = rcs_in_list_of_tfcs.index(pos)   # finds which index the rc number is in the list_of_tfcs
                    pass_fail_result = list_of_tfcs[position][2]   # reference the pass/fail result(0/1)
    #                 ##print('list_of_tfcs[position]',list_of_tfcs[position])

                    if pass_fail_result == '0' :
            
#                         print('pass_fail_result',pass_fail_result)

                        bg='red'     

                    else:
                        
#                         print('pass_fail_result',pass_fail_result)

                        bg=None  



                ############ check if pass fail 
#                 print('bg',bg)
                    
                rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , relief = RIDGE, bg = bg )
#                 ##print('rc_select',rc_select)

                rc_select.grid(row=row, column=col)

                rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it
         
        
    ########## need to do this because if not there will be a TCl error due to there being widgets of the same name ( we assign a name to each rc widget ) 
    
    except TclError:
        

        list_of_wigets_to_be_destroyed = wafer_frame.winfo_children()[1:]   # find the children of the wafer_frame window (finding the widget names of the wafer legend & the wafer map)
        
        ###### destroy the widgets of the wafer legend & wafer map
    
        for widgets in list_of_wigets_to_be_destroyed:

            widgets.destroy()

        
        
        #################### Rebuild the wafer map again  ######################
        
        if len(given_list_of_users_selected_params) > 0:                   # for the changed wafer map  (with color due to the user choosing the wafer map parameters)

            if given_upper_limit_entered != 90 or given_lower_limit_entered != 10:

                for pos , list_vals in zip(rc_list,given_list_of_users_selected_params):
                    row = pos.split(',')[0]
                    col = pos.split(',')[1]


                    try:

                        color = getcol(list_vals,given_lower_limit_entered,given_upper_limit_entered,given_list_of_users_selected_params) 

                    except ValueError:
                        
                        color = 'black'



                    rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )
    #                 ##print('rc_select',rc_select)

                    rc_select.grid(row=row, column=col)

    #                 ##print('these are -the given upper limits',given_upper_limit_entered )
    #                 ##print('these are -the given lower limits',given_lower_limit_entered )

                    rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,given_upper_limit_entered,given_lower_limit_entered,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                    tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it

            else:

                for pos , list_vals in zip(rc_list,given_list_of_users_selected_params):
                    row = pos.split(',')[0]
                    col = pos.split(',')[1]

    #                 if pos in rcs_in_list_of_tfcs:

    #                     position = rcs_in_list_of_tfcs.index(pos)   # finds which index the rc number is in the list_of_tfcs
    #                     pass_fail_result = list_of_tfcs[position][2]
    #     #                 ##print('list_of_tfcs[position]',list_of_tfcs[position])

    #                     if pass_fail_result == 'Fail' :

    #                         relief=SUNKEN     

    #                     else:

    #                         relief=RIDGE  

                    color = getcol(list_vals,given_lower_limit_entered,given_upper_limit_entered,given_list_of_users_selected_params) 

    #                 ##print('this is the rc for this ' , pos, color)

                    rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )
    #                 ##print('rc_select',rc_select)

                    rc_select.grid(row=row, column=col)

                    rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                    tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it
        
        ########## heat map 
        
        elif heat_map_check == 'Yes':
            
            ### call function


            color_list = find_heat_map_vals()
    #             #print('color_list',color_list)

            rcs_in_color_list = [i[0] for i in color_list]

            ################ make the heat map

            for pos in rc_list:
                row = pos.split(',')[0]
                col = pos.split(',')[1]

                if pos in rcs_in_color_list:

    #                     #print('pos',pos)

                    position_of_rc = rcs_in_color_list.index(pos)   # finds which index the rc number is in the list_of_tfcs
                    color = color_list[position_of_rc][1]


                rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , bg = color , relief = RIDGE )

                rc_select.grid(row=row, column=col)

#                 rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it
            
            
        ######### Pass fail map

        else:                                             # for the base wafer map (no color )
            
            string_Wafer_frame = str(wafer_frame)
            
            correct_window_name = string_Wafer_frame.split('.!')[1].split('.')[0]
            
            tfc_val,correct_mtf_limits = mtf_limits_same_window(correct_window_name)

#             tfc_val,correct_mtf_limits,window_name = get_tfc(wafer_frame)    # for 1st time users (no config file)
            ##print('this is the tfc_val',tfc_val)

            list_of_tfcs = get_list_of_tfcs(tfc_val,correct_mtf_limits,correct_window_name)   # should be same number of elements as new_final_df.rc
            ##print('list_of_tfcs',list_of_tfcs)

            rcs_in_list_of_tfcs = [i[0] for i in list_of_tfcs]

            for pos in rc_list:
                row = pos.split(',')[0]
                col = pos.split(',')[1]

                font = tkFont.Font(family="Helvetica", size=7, weight="normal")

                if pos in rcs_in_list_of_tfcs:

                    position = rcs_in_list_of_tfcs.index(pos)   # finds which index the rc number is in the list_of_tfcs
                    pass_fail_result = list_of_tfcs[position][2]
    #                 ##print('list_of_tfcs[position]',list_of_tfcs[position])

                    if pass_fail_result == '0' :

                        bg='red'     

                    else:

                        bg=None  



                ############ check if pass fail 


                rc_select = Button(wafer_frame, image=pixelVirtual, width = wafer_map_button_width ,height = wafer_map_button_height ,padx=1, pady=0, name = pos , compound = "c" , relief = RIDGE, bg = bg )
    #             ##print('rc_select',rc_select)

                rc_select.grid(row=row, column=col)

                rc_select.configure(command=lambda selected=rc_select: getgraph(selected,window,frame,canvas,90,10,window_scrollbar,wafer_frame,pixelVirtual,tooltip_1,wafer_map_button_width,wafer_map_button_height,col_max,col_min,row_min,row_max))

                tooltip_1.bind(rc_select, pos)#binding it and assigning a text to it
            
        
            
    

In [47]:
def find_heat_map_vals():
    
    
#     color_for_each_rc = []
    
    length_of_pass_list_dictionary = len(pass_list_dictionary)
    
    ####### check if the 3 windows all use the same DAT file
    
    DAT_files_selected = list(wafer_window_subfolder_name.values())[-3:]
    
    result_check =  DAT_files_selected.count(DAT_files_selected[0]) == len(DAT_files_selected)   # check if all 3 files used for the heatmap are the same

    if result_check == True:
    
    
        #### running through the passlist function

        list_of_last_3_window_names = list(pass_list_dictionary.keys())[-3:]
#         print('list_of_last_3_window_names',list_of_last_3_window_names)

        for window_name in list_of_last_3_window_names:

            tfc_val,correct_mtf_limits = mtf_limits_same_window(window_name)

            list_of_tfcs = get_list_of_tfcs(tfc_val,correct_mtf_limits,window_name)



        if length_of_pass_list_dictionary >= 3:

            ########### find the latest 3 passlists from the pass_list_dictionary

            list_of_passlists = list(pass_list_dictionary.values())[-3:]
            first_list = list_of_passlists[-3]
            second_list = list_of_passlists[-2]
            third_list = list_of_passlists[-1]
            
#             print('first_list',first_list)
#             print('second_list',second_list)
#             print('third_list',third_list)

            ############ get a list of the heatmap

            heatmaplist = heatMapVals(first_list,second_list,third_list)
#             print('heatmaplist',heatmaplist)
            
            color_for_each_rc = get_heat_map_colors(heatmaplist) # displays a list of rc and colors 

        return color_for_each_rc
    
    else:
        
        raise Exception(messagebox.showwarning("showwarning", "Warning ! Please Ensure that the files selected for the Heat Map are all the same  !" ))
        

In [48]:
def get_heat_map_colors(entire_list):
    
    heatmapvalues =  [i[1] for i in entire_list]
    print(heatmapvalues)
    
    green = Color('green')
    
    red = Color('red')
    
    cols = list(red.range_to(green,number_of_frequency_needed+1))
    
    color_list = [[entire_list[values][0],cols[i]] for values in range(len(heatmapvalues)) for i in range(number_of_frequency_needed+1) if (i/number_of_frequency_needed) == heatmapvalues[values]]
        
    return(color_list)

In [49]:
def get_list_of_tfcs(tfc_val,mtf_limits,window_number):
    
#     global passList
#     global only_unique_rcs
#     global latest_passlist
    
    number_of_interpolated_values = 101
    
    
    get_frequency_number_csv = frequency_csv_for_each_Window[window_number]  # contains the filename we will be referring to
#     print('window_number',window_number)
    
    frequency_number = get_frequency_number_csv.split('.')[0].rsplit('_',1)[1]  # gets the frequency number 


    
    ####### get the csv file data (all rows)
    
    toplevel_specific_filepath = wafer_window_subfolder_name[window_number]  # gets the correct filepath (correct filename)
    
    
    
    final_csv_df,PMO,s1_t9_list,steps_list = get_correct_csv_dataframes(toplevel_specific_filepath,window_number)  # gets the correct csv data
    
    
    ####### get correct filename to refer to (previously got the filename from the global filepath , but if users have wafer maps of diff stacks open , the TFC data would be taken from the latest stack that is in the DAT select)
    
    filename = toplevel_specific_filepath.split('-',1)[0]        # get only the parts up till no filter (eg:MAGGIE_FUAXA21B00016_no filter)
    
    filename_to_check = filename + '_' + frequency_number        # filename to pass into get_lens_pass_fail to check if the filename exists
    
    
    
    ###### call the function to get list of tfcs 
    
#     print()
    passList,tfc = get_lens_pass_fail(final_csv_df,frequency_number,number_of_interpolated_values,mtf_limits,tfc_val,steps_list,filename_to_check)
    
    new_final_df_final_df,rc_list,new_final_df_ffl_df,new_final_df_camPosDf,new_final_df_main_header,freq_list = get_correct_dataframes(toplevel_specific_filepath)  # get correct DAT data
#     #print('passList',passList)
    
    only_unique_rcs = list(dict.fromkeys(rc_list))    # remove the duplicate rcs (get all the rcs)
#     print('only_unique_rcs',only_unique_rcs)
    
    if passList == -1:
        
        current_passlist = []
        
        list_to_be_extended_onto_tfcs_list = [[rc_number,1,'0'] for rc_number in only_unique_rcs]   # populate those without a rc number with values of 0 which is fail
        
        current_passlist.extend(list_to_be_extended_onto_tfcs_list)
        
        pass_list_dictionary[window_number] = current_passlist
        
        return current_passlist
        

    else:
        
        latest_passlist = pd.DataFrame(passList).drop_duplicates(subset=[0], keep='last').values.tolist()
#         print('latest_passlist',latest_passlist)


        ##### Check if the length of the tfcs list is the same as the list of rcs , this is because for the csv in case there are measurements with just blank values , they will not be included into the csv dataframe  


        if len(latest_passlist) != len(only_unique_rcs):
            
            all_passlist_rcs = [rc[0] for rc in latest_passlist]  # get all rcs in the latest_passlist ( tfc list )

            list_of_rcs_that_are_not_in_tfcs_list = [rc for rc in only_unique_rcs if rc not in all_passlist_rcs]   # get a list of rcs that are not in the tfcs list 
#             print('list_of_rcs_that_are_not_in_tfcs_list',list_of_rcs_that_are_not_in_tfcs_list)

            list_to_be_extended_onto_tfcs_list = [[rc_number,1,'0'] for rc_number in list_of_rcs_that_are_not_in_tfcs_list]   # populate those without a rc number with values of 0 which is fail

            latest_passlist.extend(list_to_be_extended_onto_tfcs_list)

            pass_list_dictionary[window_number] = latest_passlist
            
#             print('latest_passlist',latest_passlist)

            return latest_passlist

        else:

            pass_list_dictionary[window_number] = latest_passlist
            
#             print(latest_passlist)

            return latest_passlist
    

In [50]:
def wafer_map_legend(col_max,col_min,row_min,row_max,wafer_frame,pixelVirtual,wafer_map_button_width,wafer_map_button_height):
    
    ################################################################ for the legend (row & column names)
    
    if col_min > 1:
        for rw in range(col_min-1):
            for colmn in range(1,col_max+1):
                example1 = Button(wafer_frame,image=pixelVirtual,width = wafer_map_button_width ,height = wafer_map_button_height,borderwidth=0,text = colmn,compound="c")
                example1.grid(row=rw, column=colmn)

    else:
        for rw in range(col_min):
            for colmn in range(1,col_max+1):
                example1 = Button(wafer_frame,image=pixelVirtual,width = wafer_map_button_width,height = wafer_map_button_height,borderwidth=0,text = colmn,compound="c")
                example1.grid(row=rw, column=colmn)

    
    if row_min > 1:
        for rw in range(1,row_max+1):
            for colmn in range(row_min-1):
                example2 = Button(wafer_frame,image=pixelVirtual,width = wafer_map_button_width,height = wafer_map_button_height,borderwidth=0,text = rw,compound="c")
                example2.grid(row=rw, column=colmn)

    else:
        for rw in range(1,row_max+1):
            for colmn in range(row_min):
                example2 = Button(wafer_frame,image=pixelVirtual,width = wafer_map_button_width ,height = wafer_map_button_height,borderwidth=0,text = rw,compound="c")
                example2.grid(row=rw, column=colmn)

# If we need to change the position of a whole Labelframe , just shift the labelframe position and the widgets inside should also shift

In [59]:
# from tkinter import *
from tkinter.tix import *
# from tkinter import filedialog
from tkinter import ttk
# import tkinter as tk
import os,glob
from tkinter import messagebox
import tkinter.font as tkFont
# from tqdm.tk import trange, tqdm        #tqdm

from datetime import date
from datetime import datetime
import datetime


######################################### graph plot
# %matplotlib inline
# import matplotlib.pyplot as plt   
# from matplotlib.figure import Figure
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


################################################################################################ Import for plotly 

# import sys 
import pandas as pd
# import numpy as np
# import scipy as sp
# import seaborn as sns
# import plotly.express as px
import plotly.graph_objs as go
import urllib
import plotly
import plotly.io
from plotly.io import to_image
from tqdm import tqdm

from plotly.offline import download_plotlyjs , init_notebook_mode , plot,iplot
init_notebook_mode(connected = True)
from scipy.interpolate import CubicSpline   #IMPORTING CUBIC SPLINE 
import PIL.Image
from PIL.Image import Image
import io 
# import base64
from PIL import Image, ImageTk


from colour import Color
import numpy as np
# import tkinter as tk
import math
import time
import Pmw
# import tkinter as tk
import os 
import pandas as pd
import tkinter as tk
from tkinter.ttk import *
from tkinter import *   
import tkinter.font as font
from tkinter import filedialog     # for the browse files
from tkinter import messagebox     # for warning messages
from tkinter import ttk 

################ global variables

width_of_frame = 3000                      
height_of_frame = 1000                     # ADJUST THE HEIGHT IF WE HAVE TO ADD MORE WIDGETS ( Change the height)

x_label_start_pos = 10                     # place where all the labels will be positioned according to the x axis
y_height_pos = 90                          # height the labels and the entry boxes will be positioned at for the y axis
x_widgets_pos = 150                        # x axis where all the entry boxes, dropdown , listboxes , treeview will start at 
x_browse_file_pos = 550 
y_dat_labelframe_pos = 60

x_label_pos = 5
y_widget_height_pos = 7
x_entry_pos = 120

####################### dat labelframe

x_label_pos = 5
y_pos = 15

x_csv_entry_pos = 100
x_browse_file_button_pos = 500



##################### storing_cleaned_dat_and_csv_files global 

save_location = os.getcwd()
directory_name = 'Cleaned_DAT_files'


#################### Rc lens parameters listbox

x_lens_param_listbox_pos = 120


##################### graph image 

graph_image_width = 480
graph_image_height = 380



############ browse DAT global variables 

csv_labelframe_height = '75'
csv_labelframe_width = 630


csv_dat_file_entrybox_width = '60'
csv_dat_file_entrybox_border = '3'

browse_dat_button_width = '14'
browse_dat_button_bg = 'orange'


csv_labelframe_x_position = x_label_start_pos 
csv_labelframe_y_position = 60

Dat_file_x_position = 5
Dat_file_y_position = 15

Dat_file_entry_x_position = 100
Dat_file_entry_y_position = 15

button_get_DAT_x_position = 500
button_get_DAT_y_position = 15

################## frequency_dropdown_boxes global variables

frequency_labelframe_x_position = 10
frequency_labelframe_y_position = 470
select_frequency_labelframe_height = '170'


frequency_dropdown_combobox_width = 10


frequency_dropdown_labels_x_position = 5
frequency_dropdown_combobox_x_position = 130


frequency_dropdown_1_labels_y_position = 10
frequency_dropdown_1_combobox_y_position = 10


frequency_dropdown_2_labels_y_position = 60
frequency_dropdown_2_combobox_y_position = 60

frequency_dropdown_3_labels_y_position = 110
frequency_dropdown_3_combobox_y_position = 110

heat_map_label_x_position = 250
heat_map_label_y_position = 10

heat_map_dropdown_x_position = 350
heat_map_dropdown_y_position = 10


########### display intitial info global variables

wafer_map_button_x_position = 10            # x position of the wafer map button
wafer_map_button_y_position = 480           # y position of the wafer map button


###############  display_Params global variables

param_listbox_width = 95

param_listbox_labelframe_height = '220'
param_listbox_labelframe_width = '630'

param_listbox_labelframe_x_position = 10
param_listbox_labelframe_y_position = y_dat_labelframe_pos + 180



measurement_listbox_frame_width = 150    # listbox frame width
measurement_listbox_frame_height = 100   # listbox frame height


listbox_frame_x_position = 15
listbox_frame_y_position = 15


################## display_RC_Params global variables

rc_listbox_width = 30

rc_param_listbox_labelframe_width = '220'
rc_param_listbox_labelframe_height = '235'

rc_param_listbox_labelframe_x_postion = x_browse_file_pos + 500  # placement of the labelframe
rc_param_listbox_labelframe_y_postion = y_height_pos-75

rc_listbox_frame_width = 150
rc_listbox_frame_height = 10

rc_listbox_frame_x_position = 5               # x placement of the listbox
rc_listbox_frame_y_position = 7               # y placement of the listbox 



############### wafer_param_dropdown_selection_widgets

wafer_param_dropdown_labelframe_width = "265"
wafer_param_dropdown_labelframe_height = "235"

wafer_param_dropdown_labelframe_x_position = x_browse_file_pos + 230  
wafer_param_dropdown_labelframe_y_position = y_height_pos-75

wafer_prompt_x_position = 1
wafer_prompt_y_position = 7

################ load_in_selected_config_file global variables

## Headers in the config file

mtf_specs = '[MTF Specifications]'    
lens ='[Lens Parameters]'
measurement = '[Measurement Parameters]'

#################### wafer_map_combobox global variables

wafer_map_dropdown_width = 10

wafer_map_dropdown_x_position = 169
wafer_map_dropdown_y_position = 7

wafer_map_button_width_constant = 8
wafer_map_button_height_constant = 5


############### pass_fail_toggle_combobox global variables

pass_fail_label_x_position = 1
pass_fail_label_y_position = y_widget_height_pos + 137

pass_fail_dropdown_x_position = 169
pass_fail_dropdown_y_position = y_widget_height_pos + 137


################# wafer_upper_lower_percentile global variables 

upper_lower_percentile_entrybox_width = 13

upper_limit_label_x_position = 1
upper_limit_label_y_position = y_widget_height_pos + 50

lower_limit_label_x_position = 1
lower_limit_label_y_position = y_widget_height_pos + 100

upper_limit_entry_x_position = 169
upper_limit_entry_y_position =  y_widget_height_pos + 50

lower_limit_entry_x_position = 169
lower_limit_entry_y_position = y_widget_height_pos + 100

confirm_upper_lower_limit_x_position = 5
confirm_upper_lower_limit_y_position = 186


##################  config_dropdown globals 

config_file_type = '*.kobe'    # change this

config_dropdown_width = 57

config_labelframe_height = "75"
config_labelframe_width = "630"

config_labelframe_x_position = x_label_start_pos
config_labelframe_y_position = y_dat_labelframe_pos+90

Save_file_x_position = 5
Save_file_y_position = 15


config_file_dropdown_x_position = 100
config_file_dropdown_y_position = 15

button_save_input_width = "14"
button_save_input_bg = 'Orange'

button_save_input_x_position = 500
button_save_input_y_position = 15


##################### get_config globals 

Save_file_label_x_position = x_label_start_pos
Save_file_label_y_position = 90

save_file_name_x_position = x_label_start_pos + 140
save_file_name_y_position = 90

save_file_name_width = '30'
save_file_name_border = '3'

button_get_filename_width = "15"
button_get_filename_bg = 'Orange'

button_get_filename_x_position = x_label_start_pos + 350
button_get_filename_y_position = 90


#################### input_table globals 

input_table_labelframe_height = "400"
input_table_labelframe_width = "560"

input_table_labelframe_x_position = x_browse_file_pos + 110
input_table_labelframe_y_position = y_height_pos -30

sagittal_label_x_position = 150
sagittal_label_y_position = y_pos - 10 

tangental_label_x_position = 310
tangental_label_y_position = y_pos - 10


params_x_position = 5

s_bd = '2'
s_x_position = 120


t_bd = '2'
t_x_position = 290

tfc_widget_name = "tfc"

TFC_label_x_position = 5

TFC_entry_bd = '2'
TFC_entry_x_position = x_entry_pos
TFC_entry_y_position = y_pos + 30


next_y_pos = 42

y_padding = 27


label_list_input_table = ['Camera 1' , 'Camera 2','Camera 3','Camera 4','Camera 5',                 # list of the labels
                          'Camera 6','Camera 7','Camera 8','Camera 9']



############################# creategraph1 globals 

graph_width = 480
graph_height = 460  #405

graph_layout_margin_left = 10
graph_layout_margin_right = 0
graph_layout_margin_bottom = 0
graph_layout_margin_top = 40

ffl_number_line_width = 1
ffl_number_annotation_y_position = 85

graph_x_position = x_browse_file_pos + 230
graph_y_position = y_height_pos  + 150 # 165

ffl_line_color = "blue"


###################### plot_tfc_line globals

tfc_line_color = "red"

tfc_annotation_y_position = 75
tfc_textangle = -90


################## create_empty_graph globals

# empty_graph_layout_width = 480
# empty_graph_layout_height = 460

# empty_graph_left_margin = 10
# empty_graph_right_margin = 0
# empty_graph_bottom_margin = 0
# empty_graph_top_margin = 40

empty_graph_annotation_x_position = 3
empty_graph_annotation_y_position = 60

empty_graph_x_position = x_browse_file_pos + 230
empty_graph_y_position = y_height_pos  + 165

################### get_heat_map_colors globals 

number_of_frequency_needed = 3


############## main_wafermap globals

pixelVirtual_width = 1
pixelVirtual_height = 1

Dat_filename_x_position = x_widgets_pos + 30
Dat_filename_y_position = y_pos-9

datfilename_frame_x_position = 0
datfilename_frame_y_position = 0

wafer_frame_x_position = 0
wafer_frame_y_position = 40

############### newgui globals

header_bg = "Grey"
header_fg = "Blue"
header_height = '3'

seperator_x_position = x_browse_file_pos + 100
seperator_y_position = 0
seperator_relwidth = 1
seperator_relheight = 1


#################### global main window



wafer_window = Tk()
wafer_window.state('zoomed')



#################### wafermap

# widgets_to_zoom_list = []
upper_limit = 90
lower_limit = 10



################## adding scrollbar to main window 

file_input_container = ttk.Frame(wafer_window , width = width_of_frame , height= height_of_frame)
file_input_container.pack(side="left", fill="both", expand=True)


# create the canvas that will scroll
file_input_canvas = Canvas(file_input_container,width= width_of_frame ,height= height_of_frame )

# add scrollbar to the canvas
file_input_scrollbar = ttk.Scrollbar(file_input_container, orient="vertical", command=file_input_canvas.yview)
file_input_frame = Frame(file_input_canvas,width= width_of_frame ,height= height_of_frame,name = 'file_input_frame'  )
# file_input_frame.bind(
#     "<Configure>",
#     lambda e: file_input_canvas.configure(
#         scrollregion=file_input_canvas.bbox("all")
#     )
# )
# configure the canvas to allow the user to scroll on the canvas
file_input_canvas.configure(yscrollcommand=file_input_scrollbar.set)
file_input_canvas.bind("<Configure>",lambda e: file_input_canvas.configure(scrollregion=file_input_canvas.bbox("all")))            # lambda e to pass the mouse event to the canvas


# add scrollable frame to a new window in the canvas
file_input_canvas.create_window((0, 0), window=file_input_frame, anchor="nw")
file_input_canvas.configure(yscrollcommand=file_input_scrollbar.set)


################# Scrollwheel functions (let users scroll with mouse)

def _bound_to_mousewheel(event,widget):
    widget.bind_all("<MouseWheel>",lambda event: _on_mousewheel(event,widget))

def _unbound_to_mousewheel(event,widget):
    widget.unbind_all("<MouseWheel>")

def _on_mousewheel(event,widget):
    widget.yview_scroll(int(-1*(event.delta/120)), "units")

     
################# Creating the wafer map window function        

def new_window():                 # making the wafer map windows
    
    global window
    
    global window_canvas 
    
    global window_scrollbar

#     global window_container

#     #print('final_csv_filename',final_csv_filename)
    
    window = tk.Toplevel(wafer_window)
    
#     ##print('window',window)
    window_num = str(window)
    window_number = window_num.split(".!")[-1]
    
    wafer_window_subfolder_name[window_number] = subfolder_filename  # save the file of used for each wafer window so that we can refer to the correct csv file
#     #print('wafer_window_subfolder_name',wafer_window_subfolder_name)
    
    frequency_csv_for_each_Window[window_number] = final_csv_filename
    
#     #print('frequency_number_for_each_Window',frequency_csv_for_each_Window)
    
    
    window.state('zoomed')        # Ensure tkinter window opens in full screen

    window.focus_set()            # not sure if this is needed (used to set the focus on the desired widget , in this case set the focus onto the new pop up window )
    
    
    #################### Scrollbar configuration
    
    window_container = ttk.Frame(window , width = width_of_frame , height= height_of_frame , name = 'wafer_map_window_container')
    window_container.pack(side="left", fill="both", expand=True)
    
    window_canvas = tk.Canvas(window_container, width = width_of_frame , height= height_of_frame , name = 'window_canvas')
    
    window_scrollbar = ttk.Scrollbar(window_container, orient="vertical", command=window_canvas.yview , name = 'window_scrollbar')
    window_frame =  Frame(window_canvas, width = width_of_frame , height= height_of_frame, name = 'wafer_map_window_frame')
#     window_frame.bind(
#         "<Configure>",
#         lambda e: window_canvas.configure(
#             scrollregion=window_canvas.bbox("all")
#         )
#     )
    
    # configure the canvas
    window_canvas.configure(yscrollcommand=window_scrollbar.set)
    window_canvas.bind("<Configure>",lambda e: window_canvas.configure(scrollregion=window_canvas.bbox("all")))            # lambda e to pass the mouse event to the canvas


    # add scrollable frame to a new window in the canvas
    window_canvas.create_window((0, 0), window=window_frame, anchor="nw")
    window_canvas.configure(yscrollcommand=window_scrollbar.set)
    
    ################### Calling the main function (which is to create the wafer map)
    
#     ##print('window',window)   # think shld be fine
#     ##print('window_frame',window_frame)
#     ##print('window_scrollbar',window_scrollbar)
    
    main_wafermap(window,window_frame,window_canvas,window_scrollbar)
    

########################### getting the focus from the user (tracks which window user is focused on)   
        
def focus(event,window):   
    
    global toplevel_number   # string of the toplevel name (string of window name)
    
    
    try:
        
        if str(config_labelframe.tk.call(config_file_dropdown,'state')) != 'pressed readonly':     # if user has selected a new config file then the click goes up 

            widget = window.focus_get()                                                            # if the user is not selecting a new config , we will then get the config 

            string_widget = str(widget)
            

            try:
                

                check_if_toplevel = string_widget.split('.!')[1].split('.')[0]

                if check_if_toplevel.startswith('toplevel'):

                    toplevel_number = check_if_toplevel        # if the user wants to change the config of a current wafer map window he can do so as long as he clicks on that window he wants to change first
    

            except IndexError as error:         # if the users focus is in the root window then there will be nothing to split 

                pass
            
    except (TclError, KeyError) as error:
        
        pass
    
    

############################# running the GUI




frequency_csv_for_each_Window = {} # store the frequency csv needed for each window

pass_list_dictionary = {}   # store a dictionary of previous pass lists (for heat map)

wafer_window_subfolder_name = {}     # assign the correct subfolder name to the correct window

wafer_window_parameters = {}  # assign the correct selected parameter to the correct window  (parameter -- select parameter for color)

toplevel_window_names = []   # stores the window widgets names (tracks the windows that have been opened)


def newgui():     
    
    wafer_window.counter = 0   # may have to move this to browse files
    
    file_input_scrollbar.pack(side = RIGHT , fill = Y)
    
    ######## Label of the Header 
    
    header = Label(file_input_canvas,text = "CONFIGURATION",bg= "Grey",fg = "Blue",height = '3')
    header.pack(fill="x" )   
    
    ####### binding the frames to be able to scroll (only scroll the window if users mouse is on the window)
    
    file_input_frame.bind('<Enter>',lambda event: _bound_to_mousewheel(event,file_input_canvas))
    file_input_frame.bind('<Leave>',lambda event: _unbound_to_mousewheel(event,file_input_canvas))
    
    file_input_canvas.pack(fill = "both" , expand = 1)
    
    ###### vertical line seperating in the main window
    
    sep = ttk.Separator(file_input_frame,orient='vertical')

    sep.place(x = x_browse_file_pos + 100, y=0, relwidth=1, relheight=1)
    
    ##### call widget functions
    
    browseDAT()       # display the browse csv 

    display_Params([],[])  # display an empty listbox of measurement parameters
    
    
    wafer_window.mainloop()
    


In [ ]:
newgui()

0it [00:00, ?it/s]

FILENAME_csv M-File.csv
csv_filepath C:/Users/zkob/Desktop/MTF TESTING\M-File.csv
filepath C:/Users/zkob/Desktop/MTF TESTING/M-File.dat
pmocsv                    Field  value
0                  Pitch      3
1          Max Frequency    150
2  Optimise Frequency AF     78


  0%|          | 0/1964 [00:00<?, ?it/s]

  0%|          | 0/1964 [00:00<?, ?it/s]

0it [00:00, ?it/s]

filename1 M - 78
window_number:: toplevel
wafer_map_creation toplevel_specific_filepath M-File dataframes
correct_window_name toplevel
File exists


  0%|          | 0/1964 [00:00<?, ?it/s]

filename1 M - 75
window_number:: toplevel2
wafer_map_creation toplevel_specific_filepath M-File dataframes
correct_window_name toplevel2
File exists


  0%|          | 0/1964 [00:00<?, ?it/s]